<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/biplovbhandari/tensorflow-ml-models/blob/master/Illegal_Gold_Mining_WA_TF_Training.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biplovbhandari/tensorflow-ml-models/blob/master/Illegal_Gold_Mining_WA_TF_Training.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook for developing a Deep Neural Networks (DNN) model for the Galamsey gold mining in West Africa. This notebook was developed as a part of the capacity building training to the West Africa Hub of the SERVIR.

Specifically, this notebook shows:

1.   Exporting training/testing data from Earth Engine in TFRecord format.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a simple model (Keras `Sequential` neural network and functional model) in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.
5.   Using AI Platform for inference.
6.   Using Vertex AI Platform for inference (coming soon).

# Setup software libraries

Import software libraries and/or authenticate as necessary.

In [1]:
import ee
print("Using EE version ", ee.__version__)
import folium
print("Using Folium version ", folium.__version__)

from google.api_core import exceptions, retry
import google.auth
from google.colab import auth

import io
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
from functools import partial
from pprint import pprint

import random
import requests
import tensorflow as tf
from tensorflow import keras
from keras import layers
print("Using TF version ", tf.__version__)

from typing import Dict, Iterable, List, Tuple

Using EE version  0.1.363
Using Folium version  0.14.0
Using TF version  2.12.0


## Authenticate to Colab, Cloud and Earth Engine

Identify yourself to Google Cloud, so you have access to storage and other resources.  When you run the code below, it will ask you to provide necessary permissions.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.

(You may need to run this again if you get a credentials error later.)

See [the EE auth reference](https://developers.google.com/earth-engine/guides/auth) for more info.

In [2]:
# Replace your-project with your project name.
PROJECT = "servir-ee"

BUCKET = "wa-tf-training"

auth.authenticate_user()

credentials, _ = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/earthengine",
    ]
)
ee.Initialize(
    credentials,
    project=PROJECT,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# Define variables

This set of global variables will be used throughout.  For this demo, you must have a Cloud Storage bucket into which you can write files.  ([learn more about creating Cloud Storage buckets](https://cloud.google.com/storage/docs/creating-buckets)).

In [47]:
START = '2019-01-01'
END = '2019-12-31'

# A random spot near the study area
COORDS = [-1.981041, 6.074769]
TEST_POINT = ee.Geometry.Point(COORDS)

REGION = ee.Geometry.Polygon(
        [[[-3.2689309056402194, 8.045285352640722],
          [-3.2689309056402194, 4.693792623646198],
          [0.48839331310976064, 4.693792623646198],
          [0.48839331310976064, 8.045285352640722]]])

SCALE = 30        # Meters per pixel
VALIDATION_RATIO = 0.2
TEST_RATIO = 0.2
N_CLASSES = 2 # mining/no-mining

# Predictors.
S1_BANDS = ['VV', 'VH', 's1_ratio', 's1_ndratio']

S2_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
               'B8', 'B8A', 'B9', 'B11', 'B12']

# Target variable.
OUTPUT_BANDS = ['mining']

OLD_LABEL = 'CID'

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'WA_image'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + BUCKET + '/WA_Classified.TFRecord'

# Input stack.
INPUT_BANDS = S1_BANDS + S2_BANDS
BANDS = INPUT_BANDS + OUTPUT_BANDS

EPOCHS = 20
BATCH_SIZE = 32

# Inspect and get your datasets

We will use Sentinel-2 and Sentinel-1 Dataset, plus several indices for this training. Feel free to add more dataset as you see appropriate.

In [4]:
# Helper function to set the class label (label=1)
def set_class_label(f: ee.Feature) -> ee.Feature:
    f = f.set(OUTPUT_BANDS[0], 1)
    f = f.select([OUTPUT_BANDS[0]])
    return f

In [5]:
# Helper function to set the class label to the existing one
def set_label(f: ee.Feature) -> ee.Feature:
    f = f.set(OUTPUT_BANDS[0], f.get(OLD_LABEL))
    f = f.select([OUTPUT_BANDS[0]])
    return f

In [6]:
#@title user added galamsey points
user_added_galamsey = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([-2.0554618144929204, 6.530161515380249]),
            {
              "CID": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0461569381929556, 6.296409335853294]),
            {
              "CID": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0451162410951773, 6.296563965539839]),
            {
              "CID": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.04454761278402, 6.2964946487894995]),
            {
              "CID": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0439843488908926, 6.296270702302082]),
            {
              "CID": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0427827192522208, 6.296793243955814]),
            {
              "CID": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0418600393510977, 6.297182483825538]),
            {
              "CID": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0301841865221237, 6.292543191260616]),
            {
              "CID": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0301359067598557, 6.292420552977501]),
            {
              "CID": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0294599900881027, 6.291487434657979]),
            {
              "CID": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.029583371702788, 6.290922230746797]),
            {
              "CID": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.026756323401091, 6.291055533611431]),
            {
              "CID": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.027432240072844, 6.29097555189676]),
            {
              "CID": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.028633869711516, 6.291519427313811]),
            {
              "CID": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0223923886111805, 6.291684289922669]),
            {
              "CID": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.019431229858739, 6.291854917333068]),
            {
              "CID": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.020632859497411, 6.293411889864678]),
            {
              "CID": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0216413700870106, 6.29842403001925]),
            {
              "CID": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0393010342410633, 6.29910653046785]),
            {
              "CID": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.035910722046239, 6.297592231384307]),
            {
              "CID": 1,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0241986149479385, 6.305969829490332]),
            {
              "CID": 1,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0232759350468155, 6.304946092023973]),
            {
              "CID": 1,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0223961704899307, 6.30477546891642]),
            {
              "CID": 1,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0221815937687393, 6.30065916943629]),
            {
              "CID": 1,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0222030514408584, 6.299912686757718]),
            {
              "CID": 1,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.019263350360536, 6.3028559550884795]),
            {
              "CID": 1,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0182548397709366, 6.30370907322236]),
            {
              "CID": 1,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0181260937382217, 6.302045491561733]),
            {
              "CID": 1,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0179115170170303, 6.301000418303116]),
            {
              "CID": 1,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.021751345518852, 6.3072505866614215]),
            {
              "CID": 1,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.021923006895805, 6.306333490874259]),
            {
              "CID": 1,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0191924080467327, 6.31180274663884]),
            {
              "CID": 1,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.020158003292094, 6.309925914194463]),
            {
              "CID": 1,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.016188333950053, 6.306812063553389]),
            {
              "CID": 1,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.022153566799174, 6.312997091017547]),
            {
              "CID": 1,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0142016711733257, 6.318340659785004]),
            {
              "CID": 1,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0143089595339214, 6.316506505857492]),
            {
              "CID": 1,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0193729701540386, 6.315162877344171]),
            {
              "CID": 1,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.011369258453599, 6.313136764262288]),
            {
              "CID": 1,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.014461372844074, 6.325293323765685]),
            {
              "CID": 1,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0096333966172675, 6.322691417515337]),
            {
              "CID": 1,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.005728100291584, 6.3222008927389375]),
            {
              "CID": 1,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0077665791429022, 6.32618905895267]),
            {
              "CID": 1,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0137318119920233, 6.327937870795579]),
            {
              "CID": 1,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5195873951972816, 5.880142319981529]),
            {
              "CID": 1,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5214327549995277, 5.8812095574615695]),
            {
              "CID": 1,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5118411755622718, 5.890280993327954]),
            {
              "CID": 1,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.509716866022477, 5.888744195788075]),
            {
              "CID": 1,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5065840458930824, 5.886033445320176]),
            {
              "CID": 1,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.502099392420182, 5.886737814823672]),
            {
              "CID": 1,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5004686093391273, 5.888509406900622]),
            {
              "CID": 1,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5035155987800453, 5.894790676216152]),
            {
              "CID": 1,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5053609585822914, 5.8958151972235555]),
            {
              "CID": 1,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5052751278938148, 5.89917648468263]),
            {
              "CID": 1,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4992861182221229, 5.904536119332146]),
            {
              "CID": 1,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5003375441559608, 5.903255487919286]),
            {
              "CID": 1,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5030197531708533, 5.9037463969774]),
            {
              "CID": 1,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.504049721432572, 5.902572483288904]),
            {
              "CID": 1,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.502676430416947, 5.906990654598605]),
            {
              "CID": 1,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5215046997655124, 5.927052644580127]),
            {
              "CID": 1,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5203459854710788, 5.926156239402237]),
            {
              "CID": 1,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.527255355893442, 5.920884493789381]),
            {
              "CID": 1,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5297042731401511, 5.9311986197707265]),
            {
              "CID": 1,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5479068249144023, 5.942975032689417]),
            {
              "CID": 1,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5480355709471172, 5.941929256716862]),
            {
              "CID": 1,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5461794823088115, 5.9420146262584925]),
            {
              "CID": 1,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5462974995054668, 5.9414490527985855]),
            {
              "CID": 1,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5438473548593756, 5.941428823879509]),
            {
              "CID": 1,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5442979659738776, 5.940511099636865]),
            {
              "CID": 1,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5413797225656745, 5.937331066592742]),
            {
              "CID": 1,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.541315349549317, 5.936520048515983]),
            {
              "CID": 1,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5450060691538092, 5.937523149646588]),
            {
              "CID": 1,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.53125170132544, 5.936669446672452]),
            {
              "CID": 1,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.538568767518067, 5.938099398406268]),
            {
              "CID": 1,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.526685382455335, 5.936460242154734]),
            {
              "CID": 1,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5310627475676397, 5.933557641048042]),
            {
              "CID": 1,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5452169627693402, 5.6189892403591015]),
            {
              "CID": 1,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5416979045418011, 5.619608522504382]),
            {
              "CID": 1,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5421699733284222, 5.62099656629953]),
            {
              "CID": 1,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5405391902473675, 5.624904425667992]),
            {
              "CID": 1,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.543350145294975, 5.62659141695633]),
            {
              "CID": 1,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5404299039075298, 5.629487639654139]),
            {
              "CID": 1,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.54299325182214, 5.634387258715044]),
            {
              "CID": 1,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8077063691149875, 6.3778717990748754]),
            {
              "CID": 1,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.81038857812988, 6.3789806910292155]),
            {
              "CID": 1,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8146801125537082, 6.381006545134459]),
            {
              "CID": 1,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8239288477140159, 6.381127737312764]),
            {
              "CID": 1,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8322072659522126, 6.381404958764878]),
            {
              "CID": 1,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.833344522574527, 6.37914453331921]),
            {
              "CID": 1,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8422923718482087, 6.374282452352592]),
            {
              "CID": 1,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8457899724036286, 6.373408125884726]),
            {
              "CID": 1,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.84950214968024, 6.373472101042659]),
            {
              "CID": 1,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8202985575558306, 6.370651936708877]),
            {
              "CID": 1,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8244184306027056, 6.3709078386675095]),
            {
              "CID": 1,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8271006396175982, 6.369969530862047]),
            {
              "CID": 1,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8164361765743853, 6.368860619425753]),
            {
              "CID": 1,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8296111872555376, 6.367666404430714]),
            {
              "CID": 1,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8375056158863634, 6.363371864731033]),
            {
              "CID": 1,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8381278883778185, 6.3612179876569215]),
            {
              "CID": 1,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8345659148060411, 6.360130382932083]),
            {
              "CID": 1,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.829360848646795, 6.358701044032136]),
            {
              "CID": 1,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8369354069048516, 6.355843397022889]),
            {
              "CID": 1,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8358839809710137, 6.353710814283027]),
            {
              "CID": 1,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.19419350298534, 6.017023267634075]),
            {
              "CID": 1,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1918760743964727, 6.020693641912776]),
            {
              "CID": 1,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.188099524103504, 6.012243208164909]),
            {
              "CID": 1,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.190373593744319, 6.030371457275609]),
            {
              "CID": 1,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.185180837091487, 6.033828353371626]),
            {
              "CID": 1,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.193692859385623, 6.038381563618504]),
            {
              "CID": 1,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1965252721053496, 6.04529523153077]),
            {
              "CID": 1,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1969544255477325, 6.056775139223502]),
            {
              "CID": 1,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1998430997926577, 6.061298753314106]),
            {
              "CID": 1,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1943070203859194, 6.06996179517156]),
            {
              "CID": 1,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2115095019349185, 6.057773810938727]),
            {
              "CID": 1,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1957454089707795, 6.081074242143733]),
            {
              "CID": 1,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.194415033299393, 6.086408448082177]),
            {
              "CID": 1,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2011956576890412, 6.089139541029]),
            {
              "CID": 1,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.210293131157397, 6.109946842319036]),
            {
              "CID": 1,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2006371787037837, 6.09554547586591]),
            {
              "CID": 1,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1862176230397212, 6.098319194817914]),
            {
              "CID": 1,
              "system:index": "119"
            })])

In [7]:
#@title user added non-galamsey points
user_added_non_galamsey = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([-0.5952137390609913, 6.8783537421613605]),
            {
              "CID": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8547657410141163, 6.9097110657950465]),
            {
              "CID": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5893532542879396, 6.722091397308428]),
            {
              "CID": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.590039899795752, 6.718170311448507]),
            {
              "CID": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5908982066805177, 6.713055804129979]),
            {
              "CID": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5915848521883302, 6.708793673646368]),
            {
              "CID": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6073776988680177, 6.727717247859055]),
            {
              "CID": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6075493602449709, 6.723114284071088]),
            {
              "CID": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6286637096102052, 6.72157995311995]),
            {
              "CID": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6374184398348146, 6.719193206460126]),
            {
              "CID": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.643598249405127, 6.727376289074033]),
            {
              "CID": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6547562389070802, 6.714760645875692]),
            {
              "CID": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.659906080215674, 6.718681759223226]),
            {
              "CID": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6492630748445802, 6.714590161969858]),
            {
              "CID": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6882302074129396, 6.708964159581929]),
            {
              "CID": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6937233714754396, 6.7050429679698125]),
            {
              "CID": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6966416148836427, 6.70282662831936]),
            {
              "CID": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6007554887891584, 6.760954271220445]),
            {
              "CID": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6213548540235334, 6.740838671730585]),
            {
              "CID": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5952623247266584, 6.740838671730585]),
            {
              "CID": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.615175044453221, 6.754817397298812]),
            {
              "CID": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6872728227735334, 6.736065355999499]),
            {
              "CID": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6749132036329084, 6.739815822315308]),
            {
              "CID": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5750062822461897, 6.730610080509758]),
            {
              "CID": 0,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.30806839142146547, 6.670078143604418]),
            {
              "CID": 0,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.35201370392146547, 6.656437995684539]),
            {
              "CID": 0,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.39046585235896547, 6.702812947844483]),
            {
              "CID": 0,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.44265091095271547, 6.70963242244398]),
            {
              "CID": 0,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5992060867339655, 6.7505472614957664]),
            {
              "CID": 0,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5950862136870905, 6.732817921275113]),
            {
              "CID": 0,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3928415404023498, 6.494152146780585]),
            {
              "CID": 0,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4007379637421935, 6.481530556381565]),
            {
              "CID": 0,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4189340696992248, 6.480166041190625]),
            {
              "CID": 0,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4323236571015685, 6.480166041190625]),
            {
              "CID": 0,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4172174559296935, 6.5146189176613385]),
            {
              "CID": 0,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4017679320039123, 6.51939437750446]),
            {
              "CID": 0,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4072610960664123, 6.503362296823243]),
            {
              "CID": 0,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5768071975509534, 6.4276888073703455]),
            {
              "CID": 0,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5763995017806898, 6.43160139153641]),
            {
              "CID": 0,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6252307861537618, 6.426641906535812]),
            {
              "CID": 0,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6240720718593282, 6.416065672135331]),
            {
              "CID": 0,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6232611060264013, 6.405802581998124]),
            {
              "CID": 0,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6430021643760107, 6.408532008788247]),
            {
              "CID": 0,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6708536672765972, 6.405802581998124]),
            {
              "CID": 0,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6696103527531814, 6.413754427382995]),
            {
              "CID": 0,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6783650829777907, 6.408295615090024]),
            {
              "CID": 0,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0732896961183855, 5.202873246540105]),
            {
              "CID": 0,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1017854846926043, 5.193128787050416]),
            {
              "CID": 0,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1989156513553887, 5.192103045705566]),
            {
              "CID": 0,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.228441408191326, 5.1880000636429635]),
            {
              "CID": 0,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.186706139947967, 5.2327895026585285]),
            {
              "CID": 0,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.208678796197967, 5.23860163813646]),
            {
              "CID": 0,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1606467951413677, 5.280244580550387]),
            {
              "CID": 0,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.214548467504649, 5.27033039139594]),
            {
              "CID": 0,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.21626508127418, 5.29152605120088]),
            {
              "CID": 0,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.134897588598399, 5.297337637637698]),
            {
              "CID": 0,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1249412287351177, 5.313404680225942]),
            {
              "CID": 0,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.142839675322137, 5.347533586403422]),
            {
              "CID": 0,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6360396732905391, 5.15171550001019]),
            {
              "CID": 0,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5932701964934837, 5.207587129056878]),
            {
              "CID": 0,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.552508419308869, 5.231566693976487]),
            {
              "CID": 0,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5840665933600118, 5.248690196757945]),
            {
              "CID": 0,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.567758762549465, 5.281818592131666]),
            {
              "CID": 0,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6094928169666844, 5.299711795594893]),
            {
              "CID": 0,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6249361175795207, 5.328223652182886]),
            {
              "CID": 0,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6079796595590912, 5.364395603590143]),
            {
              "CID": 0,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6432273105581885, 5.377555496407703]),
            {
              "CID": 0,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6245162204702979, 5.3866134391743365]),
            {
              "CID": 0,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.648033829112876, 5.399089252441305]),
            {
              "CID": 0,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.028873500340552, 5.9660891248388355]),
            {
              "CID": 0,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.0254402728014895, 5.942869137801076]),
            {
              "CID": 0,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.020633754246802, 5.9216971192806405]),
            {
              "CID": 0,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.0075874895983645, 5.8895960604098745]),
            {
              "CID": 0,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.872318324559302, 5.914867262638918]),
            {
              "CID": 0,
              "system:index": "73"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.8524056048327395, 5.940820268328766]),
            {
              "CID": 0,
              "system:index": "74"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.821506556981177, 5.9216971192806405]),
            {
              "CID": 0,
              "system:index": "75"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.729496058934302, 5.927843918061265]),
            {
              "CID": 0,
              "system:index": "76"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.015140590184302, 6.0241347818736894]),
            {
              "CID": 0,
              "system:index": "77"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.998661097996802, 6.0302804303597295]),
            {
              "CID": 0,
              "system:index": "78"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.5296921151729745, 6.094791119592002]),
            {
              "CID": 0,
              "system:index": "79"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.5307220834346933, 6.087622046431368]),
            {
              "CID": 0,
              "system:index": "80"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.523168982848756, 6.085573722237282]),
            {
              "CID": 0,
              "system:index": "81"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.5159592050167245, 6.095815265084456]),
            {
              "CID": 0,
              "system:index": "82"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4871200936885995, 6.116297763948284]),
            {
              "CID": 0,
              "system:index": "83"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.479223670348756, 6.12414918062657]),
            {
              "CID": 0,
              "system:index": "84"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.493299903258912, 6.110153096554092]),
            {
              "CID": 0,
              "system:index": "85"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.499823035583131, 6.09922906927119]),
            {
              "CID": 0,
              "system:index": "86"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.566084327087037, 6.125173269946752]),
            {
              "CID": 0,
              "system:index": "87"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.522139014587037, 6.1320004818669425]),
            {
              "CID": 0,
              "system:index": "88"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4373382943721933, 6.116639132289249]),
            {
              "CID": 0,
              "system:index": "89"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4239487069698495, 6.1197114375485295]),
            {
              "CID": 0,
              "system:index": "90"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0938895297322957, 6.062601275340641]),
            {
              "CID": 0,
              "system:index": "91"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.106592471626827, 6.030167056790679]),
            {
              "CID": 0,
              "system:index": "92"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.123415286568233, 6.006608355131784]),
            {
              "CID": 0,
              "system:index": "93"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1251319003377644, 5.994657899176307]),
            {
              "CID": 0,
              "system:index": "94"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.114832217720577, 5.979292642693575]),
            {
              "CID": 0,
              "system:index": "95"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0664237094197957, 5.966658663991108]),
            {
              "CID": 0,
              "system:index": "96"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.042047793892452, 5.981682822076423]),
            {
              "CID": 0,
              "system:index": "97"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.060243899849483, 6.014802801954523]),
            {
              "CID": 0,
              "system:index": "98"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.080499942329952, 6.050652052717796]),
            {
              "CID": 0,
              "system:index": "99"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.9799063754354207, 6.036995474862368]),
            {
              "CID": 0,
              "system:index": "100"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0770667147908894, 6.03323985556724]),
            {
              "CID": 0,
              "system:index": "101"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0911429477010457, 6.016509962838818]),
            {
              "CID": 0,
              "system:index": "102"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.142641360786983, 6.048603588015978]),
            {
              "CID": 0,
              "system:index": "103"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1556876254354207, 6.043482392315169]),
            {
              "CID": 0,
              "system:index": "104"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.9023154330526082, 6.047237940569912]),
            {
              "CID": 0,
              "system:index": "105"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.9126151156697957, 6.067380890310294]),
            {
              "CID": 0,
              "system:index": "106"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0465334268093383, 6.29464755831165]),
            {
              "CID": 0,
              "system:index": "107"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0467050881862914, 6.294050365340061]),
            {
              "CID": 0,
              "system:index": "108"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0462759347439086, 6.29381575362766]),
            {
              "CID": 0,
              "system:index": "109"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.046941122579602, 6.292941290857137]),
            {
              "CID": 0,
              "system:index": "110"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.025696870117528, 6.293091964382866]),
            {
              "CID": 0,
              "system:index": "111"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0389791691592762, 6.29716566746482]),
            {
              "CID": 0,
              "system:index": "112"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0356225401152805, 6.30172988321218]),
            {
              "CID": 0,
              "system:index": "113"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.027052485339784, 6.30196017953789]),
            {
              "CID": 0,
              "system:index": "114"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0279966229130264, 6.300872450004357]),
            {
              "CID": 0,
              "system:index": "115"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0276962155033584, 6.299848702478158]),
            {
              "CID": 0,
              "system:index": "116"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.027031027667665, 6.298718312238765]),
            {
              "CID": 0,
              "system:index": "117"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.025936686389589, 6.2992301873691465]),
            {
              "CID": 0,
              "system:index": "118"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0282970303226944, 6.304114303844404]),
            {
              "CID": 0,
              "system:index": "119"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0290695065189834, 6.304796796807937]),
            {
              "CID": 0,
              "system:index": "120"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.029584490649843, 6.3056499117492155]),
            {
              "CID": 0,
              "system:index": "121"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0292411678959366, 6.306183107874757]),
            {
              "CID": 0,
              "system:index": "122"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0272456043888565, 6.304562189953032]),
            {
              "CID": 0,
              "system:index": "123"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0310585007286175, 6.308731562735626]),
            {
              "CID": 0,
              "system:index": "124"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0249430641746624, 6.309691309660867]),
            {
              "CID": 0,
              "system:index": "125"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.027324865779887, 6.312250626103606]),
            {
              "CID": 0,
              "system:index": "126"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.019136935760728, 6.316975708643714]),
            {
              "CID": 0,
              "system:index": "127"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0170555415651714, 6.3159946477874405]),
            {
              "CID": 0,
              "system:index": "128"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.016218692352525, 6.320345425245967]),
            {
              "CID": 0,
              "system:index": "129"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0100388827822124, 6.3202814434895656]),
            {
              "CID": 0,
              "system:index": "130"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.006906062652818, 6.316378541387433]),
            {
              "CID": 0,
              "system:index": "131"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.021456573954914, 6.323437867369224]),
            {
              "CID": 0,
              "system:index": "132"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0231946453965644, 6.32388573676429]),
            {
              "CID": 0,
              "system:index": "133"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.017336700908039, 6.328854928298621]),
            {
              "CID": 0,
              "system:index": "134"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.0033462986863593, 6.328705639978828]),
            {
              "CID": 0,
              "system:index": "135"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.001479481211994, 6.323736447009037]),
            {
              "CID": 0,
              "system:index": "136"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.615395460620915, 5.506909809148459]),
            {
              "CID": 0,
              "system:index": "137"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6215752701912276, 5.475469065302857]),
            {
              "CID": 0,
              "system:index": "138"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6335915665779464, 5.444368441606862]),
            {
              "CID": 0,
              "system:index": "139"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.582436476245915, 5.410531869984839]),
            {
              "CID": 0,
              "system:index": "140"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.558747206226384, 5.474785552469465]),
            {
              "CID": 0,
              "system:index": "141"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.541237745777165, 5.553726092276223]),
            {
              "CID": 0,
              "system:index": "142"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.5251015763435714, 5.5639773377867625]),
            {
              "CID": 0,
              "system:index": "143"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4979790787849776, 5.505884593684815]),
            {
              "CID": 0,
              "system:index": "144"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4801262955818526, 5.4467608545179775]),
            {
              "CID": 0,
              "system:index": "145"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.406311903492009, 5.384555035226708]),
            {
              "CID": 0,
              "system:index": "146"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.524414930835759, 5.397543591703738]),
            {
              "CID": 0,
              "system:index": "147"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.524758253589665, 5.471026217918734]),
            {
              "CID": 0,
              "system:index": "148"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.421418104663884, 5.525021657084106]),
            {
              "CID": 0,
              "system:index": "149"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.323914442554509, 5.423519869408556]),
            {
              "CID": 0,
              "system:index": "150"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.355500135913884, 5.468633901812101]),
            {
              "CID": 0,
              "system:index": "151"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.36133662273029, 5.53014755131382]),
            {
              "CID": 0,
              "system:index": "152"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6421746354256026, 5.5219460992513945]),
            {
              "CID": 0,
              "system:index": "153"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.652130995288884, 5.426254149481544]),
            {
              "CID": 0,
              "system:index": "154"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6943596940193526, 5.497341062751895]),
            {
              "CID": 0,
              "system:index": "155"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7050026993904464, 5.44231779439061]),
            {
              "CID": 0,
              "system:index": "156"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.701569471851384, 5.418734849474389]),
            {
              "CID": 0,
              "system:index": "157"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.691398535266911, 5.41997383148843]),
            {
              "CID": 0,
              "system:index": "158"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6943167786751143, 5.421597321314111]),
            {
              "CID": 0,
              "system:index": "159"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.689907090182937, 5.386533511762839]),
            {
              "CID": 0,
              "system:index": "160"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.689177529330886, 5.3852090095127805]),
            {
              "CID": 0,
              "system:index": "161"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.691881196017898, 5.391190609650954]),
            {
              "CID": 0,
              "system:index": "162"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6910658044773705, 5.389823391955575]),
            {
              "CID": 0,
              "system:index": "163"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.689778344150222, 5.396189473135512]),
            {
              "CID": 0,
              "system:index": "164"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7036829156834252, 5.388755250989233]),
            {
              "CID": 0,
              "system:index": "165"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.684285180087722, 5.381961830495537]),
            {
              "CID": 0,
              "system:index": "166"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6942666351445865, 5.366058938739399]),
            {
              "CID": 0,
              "system:index": "167"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.686563330853815, 5.364777119311377]),
            {
              "CID": 0,
              "system:index": "168"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7105263789041123, 5.346017645442747]),
            {
              "CID": 0,
              "system:index": "169"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.716802747998961, 5.345024203045636]),
            {
              "CID": 0,
              "system:index": "170"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.736384887953387, 5.325961533321956]),
            {
              "CID": 0,
              "system:index": "171"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7448987429443172, 5.307299601012811]),
            {
              "CID": 0,
              "system:index": "172"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.746083314305079, 5.298783247812866]),
            {
              "CID": 0,
              "system:index": "173"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7593908826353686, 5.2912448075766845]),
            {
              "CID": 0,
              "system:index": "174"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.702868325587069, 5.2460578703731375]),
            {
              "CID": 0,
              "system:index": "175"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6896503995616783, 5.232040442398229]),
            {
              "CID": 0,
              "system:index": "176"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.676775796290194, 5.222980230351897]),
            {
              "CID": 0,
              "system:index": "177"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6745441983898033, 5.267254358225668]),
            {
              "CID": 0,
              "system:index": "178"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7100781034191, 5.273066172283117]),
            {
              "CID": 0,
              "system:index": "179"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7476719449718345, 5.263322806129684]),
            {
              "CID": 0,
              "system:index": "180"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.738573891993319, 5.2279377211073195]),
            {
              "CID": 0,
              "system:index": "181"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6776393413986943, 5.144037107600932]),
            {
              "CID": 0,
              "system:index": "182"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7123149395432256, 5.150704902728141]),
            {
              "CID": 0,
              "system:index": "183"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.707336759611585, 5.137198270804092]),
            {
              "CID": 0,
              "system:index": "184"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6108630657639287, 5.17156268160156]),
            {
              "CID": 0,
              "system:index": "185"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6468690395798466, 5.163570130185731]),
            {
              "CID": 0,
              "system:index": "186"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.6474698543991826, 5.161817739130393]),
            {
              "CID": 0,
              "system:index": "187"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4436224177190624, 5.015603352433459]),
            {
              "CID": 0,
              "system:index": "188"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4688286216786737, 5.023483889913042]),
            {
              "CID": 0,
              "system:index": "189"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.459258499913537, 5.0192943272804005]),
            {
              "CID": 0,
              "system:index": "190"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.454409066014611, 5.019208825721959]),
            {
              "CID": 0,
              "system:index": "191"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.437285843663537, 5.023483889913042]),
            {
              "CID": 0,
              "system:index": "192"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.4299044044545526, 5.018353809521185]),
            {
              "CID": 0,
              "system:index": "193"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.475609246068322, 5.032717932844358]),
            {
              "CID": 0,
              "system:index": "194"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.443980637364709, 5.031093434784778]),
            {
              "CID": 0,
              "system:index": "195"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6963969427700167, 5.541461697504382]),
            {
              "CID": 0,
              "system:index": "196"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6970835882778292, 5.536848482949395]),
            {
              "CID": 0,
              "system:index": "197"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7136489111538058, 5.5224105974472995]),
            {
              "CID": 0,
              "system:index": "198"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7193995672817355, 5.518480697848309]),
            {
              "CID": 0,
              "system:index": "199"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.673394318258298, 5.54667287800588]),
            {
              "CID": 0,
              "system:index": "200"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6780291754360324, 5.543426574243814]),
            {
              "CID": 0,
              "system:index": "201"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7061816412563449, 5.5519694405303825]),
            {
              "CID": 0,
              "system:index": "202"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7286692816372042, 5.53821536509118]),
            {
              "CID": 0,
              "system:index": "203"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7533885199184542, 5.5424014219781474]),
            {
              "CID": 0,
              "system:index": "204"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7624865728969699, 5.556326254674316]),
            {
              "CID": 0,
              "system:index": "205"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7849262012403435, 5.525657016590641]),
            {
              "CID": 0,
              "system:index": "206"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8214042438428826, 5.533345833034325]),
            {
              "CID": 0,
              "system:index": "207"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8209750904004998, 5.528732555154062]),
            {
              "CID": 0,
              "system:index": "208"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8257816089551873, 5.508912137790584]),
            {
              "CID": 0,
              "system:index": "209"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8053539050977654, 5.50583649645685]),
            {
              "CID": 0,
              "system:index": "210"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8498441176069793, 5.5501408186816885]),
            {
              "CID": 0,
              "system:index": "211"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8549939589155731, 5.556804189124935]),
            {
              "CID": 0,
              "system:index": "212"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8304463820112762, 5.579698271454425]),
            {
              "CID": 0,
              "system:index": "213"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8302747206343231, 5.584738266695696]),
            {
              "CID": 0,
              "system:index": "214"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7888308872865801, 5.605757897778883]),
            {
              "CID": 0,
              "system:index": "215"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7828227390932208, 5.608405918306628]),
            {
              "CID": 0,
              "system:index": "216"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7911483158754473, 5.614641532068428]),
            {
              "CID": 0,
              "system:index": "217"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7623092045473223, 5.600376399700733]),
            {
              "CID": 0,
              "system:index": "218"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8376981328689026, 5.629448257120339]),
            {
              "CID": 0,
              "system:index": "219"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8396722387038635, 5.633889910461831]),
            {
              "CID": 0,
              "system:index": "220"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8800555776320862, 5.631028464603849]),
            {
              "CID": 0,
              "system:index": "221"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8798839162551331, 5.633847202418313]),
            {
              "CID": 0,
              "system:index": "222"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8788539479934143, 5.636623218713945]),
            {
              "CID": 0,
              "system:index": "223"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8822871755324768, 5.638160698950714]),
            {
              "CID": 0,
              "system:index": "224"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8771373342238831, 5.639698175114491]),
            {
              "CID": 0,
              "system:index": "225"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8853770803176331, 5.631455546969933]),
            {
              "CID": 0,
              "system:index": "226"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.857138783808844, 5.6292347152389945]),
            {
              "CID": 0,
              "system:index": "227"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3362344083351219, 5.558396540717861]),
            {
              "CID": 0,
              "system:index": "228"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.1439736661476219, 5.915029797683091]),
            {
              "CID": 0,
              "system:index": "229"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5303591466010902, 5.88065459422749]),
            {
              "CID": 0,
              "system:index": "230"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5278700566352699, 5.877367492952719]),
            {
              "CID": 0,
              "system:index": "231"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5161970830024574, 5.876855215680662]),
            {
              "CID": 0,
              "system:index": "232"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.522248146540055, 5.870323639150117]),
            {
              "CID": 0,
              "system:index": "233"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5328911519111488, 5.868487953529722]),
            {
              "CID": 0,
              "system:index": "234"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.548941490656266, 5.868658715237822]),
            {
              "CID": 0,
              "system:index": "235"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5487269139350746, 5.864816564155595]),
            {
              "CID": 0,
              "system:index": "236"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5396717763007972, 5.88402705456899]),
            {
              "CID": 0,
              "system:index": "237"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.508598132902994, 5.904941040240273]),
            {
              "CID": 0,
              "system:index": "238"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5052721937245273, 5.906243011523601]),
            {
              "CID": 0,
              "system:index": "239"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.504478259856119, 5.91042637394399]),
            {
              "CID": 0,
              "system:index": "240"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5123317678517245, 5.909572629078617]),
            {
              "CID": 0,
              "system:index": "241"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.508726878935709, 5.913030287648702]),
            {
              "CID": 0,
              "system:index": "242"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5239079590428561, 5.917661660963335]),
            {
              "CID": 0,
              "system:index": "243"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5144665833104343, 5.918045840697438]),
            {
              "CID": 0,
              "system:index": "244"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5358411673662253, 5.9308998204519465]),
            {
              "CID": 0,
              "system:index": "245"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5485517267965787, 5.9315351806950485]),
            {
              "CID": 0,
              "system:index": "246"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.1086276617178847, 5.490985233673028]),
            {
              "CID": 0,
              "system:index": "247"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.1038211431631972, 5.471847075892822]),
            {
              "CID": 0,
              "system:index": "248"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.2026980962881972, 5.485175500636095]),
            {
              "CID": 0,
              "system:index": "249"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.2095645513663222, 5.455442453162348]),
            {
              "CID": 0,
              "system:index": "250"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.2926486578116347, 5.473897622126666]),
            {
              "CID": 0,
              "system:index": "251"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.2912753667960097, 5.441088040248146]),
            {
              "CID": 0,
              "system:index": "252"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0131839361319472, 5.432885364725516]),
            {
              "CID": 0,
              "system:index": "253"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.027946814549916, 5.411352810517349]),
            {
              "CID": 0,
              "system:index": "254"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0674289312491347, 5.481074478546664]),
            {
              "CID": 0,
              "system:index": "255"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0536960210928847, 5.464328346243368]),
            {
              "CID": 0,
              "system:index": "256"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0701755132803847, 5.441771591495003]),
            {
              "CID": 0,
              "system:index": "257"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0972980108389785, 5.416821468454364]),
            {
              "CID": 0,
              "system:index": "258"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3439856724177712, 5.436379341371542]),
            {
              "CID": 0,
              "system:index": "259"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3498221592341775, 5.472606769081495]),
            {
              "CID": 0,
              "system:index": "260"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3910208897029275, 5.468505660733946]),
            {
              "CID": 0,
              "system:index": "261"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3432990269099587, 5.527627244521317]),
            {
              "CID": 0,
              "system:index": "262"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5333602558551185, 5.486465984986711]),
            {
              "CID": 0,
              "system:index": "263"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5223739277301185, 5.487491233887653]),
            {
              "CID": 0,
              "system:index": "264"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5242622028766029, 5.481168837600732]),
            {
              "CID": 0,
              "system:index": "265"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5568778644976966, 5.487491233887653]),
            {
              "CID": 0,
              "system:index": "266"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5537879597125404, 5.501161050693716]),
            {
              "CID": 0,
              "system:index": "267"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5661475788531654, 5.485697047154597]),
            {
              "CID": 0,
              "system:index": "268"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.534218562739884, 5.476384276987439]),
            {
              "CID": 0,
              "system:index": "269"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.503834499019181, 5.475188130847583]),
            {
              "CID": 0,
              "system:index": "270"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.486754192012345, 5.47749498197363]),
            {
              "CID": 0,
              "system:index": "271"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.553542502562375, 5.540776862082961]),
            {
              "CID": 0,
              "system:index": "272"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5569757301014375, 5.5226654957980985]),
            {
              "CID": 0,
              "system:index": "273"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5452169257801485, 5.559314808061195]),
            {
              "CID": 0,
              "system:index": "274"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5516305181571544, 5.6133030646275115]),
            {
              "CID": 0,
              "system:index": "275"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.546394846160084, 5.604205858899028]),
            {
              "CID": 0,
              "system:index": "276"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5464806768485606, 5.601942212908025]),
            {
              "CID": 0,
              "system:index": "277"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5390134069510997, 5.610569646829449]),
            {
              "CID": 0,
              "system:index": "278"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5639043066093028, 5.614157255061282]),
            {
              "CID": 0,
              "system:index": "279"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5397102279879116, 5.634109656613752]),
            {
              "CID": 0,
              "system:index": "280"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.532135669729855, 5.635241418194397]),
            {
              "CID": 0,
              "system:index": "281"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5345603866793178, 5.632209146356952]),
            {
              "CID": 0,
              "system:index": "282"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5372640533663295, 5.6356684974629125]),
            {
              "CID": 0,
              "system:index": "283"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.533659164450314, 5.636586716826434]),
            {
              "CID": 0,
              "system:index": "284"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5560395164705776, 5.630799775185508]),
            {
              "CID": 0,
              "system:index": "285"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5458471222139858, 5.632764352243423]),
            {
              "CID": 0,
              "system:index": "286"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.545417968771603, 5.63675754817568]),
            {
              "CID": 0,
              "system:index": "287"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7716669539595085, 6.8067028241732634]),
            {
              "CID": 0,
              "system:index": "288"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.769607017436071, 6.778747973933035]),
            {
              "CID": 0,
              "system:index": "289"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.7620539168501335, 6.737835516342394]),
            {
              "CID": 0,
              "system:index": "290"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.654250572123571, 6.840110143417861]),
            {
              "CID": 0,
              "system:index": "291"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.681716392436071, 6.9246406518037205]),
            {
              "CID": 0,
              "system:index": "292"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.5848993758345085, 7.035052827414389]),
            {
              "CID": 0,
              "system:index": "293"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.8677506463691005, 7.063188764951801]),
            {
              "CID": 0,
              "system:index": "294"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.9020829217597255, 7.035930576456779]),
            {
              "CID": 0,
              "system:index": "295"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.045451347150838, 7.017695430898716]),
            {
              "CID": 0,
              "system:index": "296"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.042533103742635, 7.025873376780326]),
            {
              "CID": 0,
              "system:index": "297"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.0102607648754476, 7.046317611766523]),
            {
              "CID": 0,
              "system:index": "298"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.988803092756307, 7.062331632859289]),
            {
              "CID": 0,
              "system:index": "299"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.984511558332479, 7.061650196440183]),
            {
              "CID": 0,
              "system:index": "300"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.0363384147788253, 6.9573321701337445]),
            {
              "CID": 0,
              "system:index": "301"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.0380550285483565, 6.956565381338016]),
            {
              "CID": 0,
              "system:index": "302"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.089942015514986, 6.935974344878393]),
            {
              "CID": 0,
              "system:index": "303"
            }),
        ee.Feature(
            ee.Geometry.Point([-3.069170988903658, 6.92583513173857]),
            {
              "CID": 0,
              "system:index": "304"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.48767876987931613, 7.031112050919144]),
            {
              "CID": 0,
              "system:index": "305"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4938439951699247, 7.085087854520407]),
            {
              "CID": 0,
              "system:index": "306"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.49217029674463175, 7.083256582112781]),
            {
              "CID": 0,
              "system:index": "307"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.49354358776025675, 7.077081307600452]),
            {
              "CID": 0,
              "system:index": "308"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5028991328042021, 7.078486722168703]),
            {
              "CID": 0,
              "system:index": "309"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4923419581215849, 7.091603718266655]),
            {
              "CID": 0,
              "system:index": "310"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.48045440776758097, 7.085300792700362]),
            {
              "CID": 0,
              "system:index": "311"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5276183710854521, 7.075803654278964]),
            {
              "CID": 0,
              "system:index": "312"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5310515986245146, 7.072396561467149]),
            {
              "CID": 0,
              "system:index": "313"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5237559901040068, 7.090368692347058]),
            {
              "CID": 0,
              "system:index": "314"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5297641382973661, 7.093264609971048]),
            {
              "CID": 0,
              "system:index": "315"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5548266993325224, 7.084917503905642]),
            {
              "CID": 0,
              "system:index": "316"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5580024348061552, 7.078784839860151]),
            {
              "CID": 0,
              "system:index": "317"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.42282048321317767, 7.063909098408237]),
            {
              "CID": 0,
              "system:index": "318"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4326051816995058, 7.0608426370750035]),
            {
              "CID": 0,
              "system:index": "319"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.44693890667509173, 7.039291655323156]),
            {
              "CID": 0,
              "system:index": "320"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4326051816995058, 7.037502793623342]),
            {
              "CID": 0,
              "system:index": "321"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4195589170510683, 7.041591610231921]),
            {
              "CID": 0,
              "system:index": "322"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.427906178195121, 6.996463789515037]),
            {
              "CID": 0,
              "system:index": "323"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.42550291891777725, 6.986240689848573]),
            {
              "CID": 0,
              "system:index": "324"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.44103827353203506, 6.9956118730881975]),
            {
              "CID": 0,
              "system:index": "325"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4400319622510307, 6.954150973797295]),
            {
              "CID": 0,
              "system:index": "326"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4388303326123588, 6.947505384318949]),
            {
              "CID": 0,
              "system:index": "327"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.37943549618657757, 6.980391612821304]),
            {
              "CID": 0,
              "system:index": "328"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3777188824170463, 6.985503254556358]),
            {
              "CID": 0,
              "system:index": "329"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3900500803529017, 6.832646594709071]),
            {
              "CID": 0,
              "system:index": "330"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3783771067200892, 6.762419262248422]),
            {
              "CID": 0,
              "system:index": "331"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5437803339365277, 6.7752989662658765]),
            {
              "CID": 0,
              "system:index": "332"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5049848627451214, 6.7579114706398276]),
            {
              "CID": 0,
              "system:index": "333"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5652380060556683, 6.799163135351095]),
            {
              "CID": 0,
              "system:index": "334"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6057500910166058, 6.811265225622441]),
            {
              "CID": 0,
              "system:index": "335"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6167364191416058, 6.807344863589096]),
            {
              "CID": 0,
              "system:index": "336"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4593229364755902, 6.745126148380371]),
            {
              "CID": 0,
              "system:index": "337"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5187177729013714, 6.7296128376703646]),
            {
              "CID": 0,
              "system:index": "338"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5370855402353558, 6.715633385865732]),
            {
              "CID": 0,
              "system:index": "339"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6330442499521527, 6.735920507376409]),
            {
              "CID": 0,
              "system:index": "340"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6747436076780322, 6.763238742704393]),
            {
              "CID": 0,
              "system:index": "341"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7028960734983447, 6.753862976003982]),
            {
              "CID": 0,
              "system:index": "342"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7967495852267881, 6.72083559117631]),
            {
              "CID": 0,
              "system:index": "343"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.818550580099835, 6.714868693433871]),
            {
              "CID": 0,
              "system:index": "344"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8427548342502256, 6.687420022470001]),
            {
              "CID": 0,
              "system:index": "345"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.73477982814671, 6.691852831656653]),
            {
              "CID": 0,
              "system:index": "346"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7462477845938142, 6.672350110313261]),
            {
              "CID": 0,
              "system:index": "347"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7435012025625642, 6.631599232473835]),
            {
              "CID": 0,
              "system:index": "348"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7400679750235017, 6.629894101230372]),
            {
              "CID": 0,
              "system:index": "349"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.68952161923856, 6.629002838356383]),
            {
              "CID": 0,
              "system:index": "350"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7111509527346538, 6.603936684001998]),
            {
              "CID": 0,
              "system:index": "351"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.732093640722935, 6.591658920386866]),
            {
              "CID": 0,
              "system:index": "352"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.82534874332968, 6.351658135130856]),
            {
              "CID": 0,
              "system:index": "353"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8420857275826097, 6.383560775933822]),
            {
              "CID": 0,
              "system:index": "354"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8168515051705003, 6.387058003567226]),
            {
              "CID": 0,
              "system:index": "355"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8104142035347581, 6.382451893892063]),
            {
              "CID": 0,
              "system:index": "356"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8257861424348953, 6.350770179825258]),
            {
              "CID": 0,
              "system:index": "357"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8232112217805985, 6.354054372337765]),
            {
              "CID": 0,
              "system:index": "358"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8328028012178543, 6.351452611163088]),
            {
              "CID": 0,
              "system:index": "359"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8250136662386063, 6.3595643124906225]),
            {
              "CID": 0,
              "system:index": "360"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8278846040861807, 6.362929921347485]),
            {
              "CID": 0,
              "system:index": "361"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8262752786772452, 6.3680545718955885]),
            {
              "CID": 0,
              "system:index": "362"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8255457178251944, 6.372980687115024]),
            {
              "CID": 0,
              "system:index": "363"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8178853288786612, 6.372447560044337]),
            {
              "CID": 0,
              "system:index": "364"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8193444505827627, 6.368481077247375]),
            {
              "CID": 0,
              "system:index": "365"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8385276094572744, 6.367862644371681]),
            {
              "CID": 0,
              "system:index": "366"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.838913847555419, 6.375539689340997]),
            {
              "CID": 0,
              "system:index": "367"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8526038423674307, 6.372340934563732]),
            {
              "CID": 0,
              "system:index": "368"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8508013979094229, 6.368673004540082]),
            {
              "CID": 0,
              "system:index": "369"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8524107233183584, 6.3779067550303745]),
            {
              "CID": 0,
              "system:index": "370"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8531617418425284, 6.382512905577688]),
            {
              "CID": 0,
              "system:index": "371"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.83814137135913, 6.380529706945849]),
            {
              "CID": 0,
              "system:index": "372"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2084859578256966, 5.910917343030686]),
            {
              "CID": 0,
              "system:index": "373"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.221188899720228, 5.920735305561556]),
            {
              "CID": 0,
              "system:index": "374"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.218013164246595, 5.906819359303706]),
            {
              "CID": 0,
              "system:index": "375"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2575811116342903, 5.934309418503252]),
            {
              "CID": 0,
              "system:index": "376"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2734597890024544, 5.915698285690933]),
            {
              "CID": 0,
              "system:index": "377"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1949247090463997, 5.908356106756096]),
            {
              "CID": 0,
              "system:index": "378"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1962980000620247, 5.902806720854084]),
            {
              "CID": 0,
              "system:index": "379"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1883157460337044, 5.934394789225799]),
            {
              "CID": 0,
              "system:index": "380"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.22487961932472, 5.899647814825521]),
            {
              "CID": 0,
              "system:index": "381"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.254147884095228, 5.915356791157096]),
            {
              "CID": 0,
              "system:index": "382"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.262130138123548, 5.927821204916799]),
            {
              "CID": 0,
              "system:index": "383"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8950523213716797, 5.042510960970062]),
            {
              "CID": 0,
              "system:index": "384"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8538535909029297, 5.143733245525184]),
            {
              "CID": 0,
              "system:index": "385"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7453636006685547, 5.192970718963428]),
            {
              "CID": 0,
              "system:index": "386"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.1287559346529297, 5.321517013424059]),
            {
              "CID": 0,
              "system:index": "387"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.1562217549654297, 5.268187199601469]),
            {
              "CID": 0,
              "system:index": "388"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8593467549654297, 5.497882440520892]),
            {
              "CID": 0,
              "system:index": "389"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.3290122823091797, 5.272973396177501]),
            {
              "CID": 0,
              "system:index": "390"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8387473897310547, 5.534106134187922]),
            {
              "CID": 0,
              "system:index": "391"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.199106967450055, 6.071045791785667]),
            {
              "CID": 0,
              "system:index": "392"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.20073775053111, 6.077276255267498]),
            {
              "CID": 0,
              "system:index": "393"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2149772948320168, 6.092230917891292]),
            {
              "CID": 0,
              "system:index": "394"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.2123165434892433, 6.090267957538495]),
            {
              "CID": 0,
              "system:index": "395"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1794879090425345, 6.092400014919883]),
            {
              "CID": 0,
              "system:index": "396"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7507071214210148, 6.979619394049918]),
            {
              "CID": 0,
              "system:index": "397"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7398066239844914, 6.97910822596132]),
            {
              "CID": 0,
              "system:index": "398"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7313952165137882, 6.9860089480377185]),
            {
              "CID": 0,
              "system:index": "399"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7164606767188664, 6.980130561580213]),
            {
              "CID": 0,
              "system:index": "400"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.810521049705033, 6.979532933959761]),
            {
              "CID": 0,
              "system:index": "401"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6311591811688233, 6.975918359425315]),
            {
              "CID": 0,
              "system:index": "402"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6260093398602296, 6.9489958690310045]),
            {
              "CID": 0,
              "system:index": "403"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5880281584366176, 7.095257425127431]),
            {
              "CID": 0,
              "system:index": "404"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0097461882137138, 7.102592199601432]),
            {
              "CID": 0,
              "system:index": "405"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0187682774662399, 7.130604445765792]),
            {
              "CID": 0,
              "system:index": "406"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.9183463719486618, 6.970693498035937]),
            {
              "CID": 0,
              "system:index": "407"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.9066733983158493, 6.936272947538732]),
            {
              "CID": 0,
              "system:index": "408"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0515556004642868, 6.916505552851457]),
            {
              "CID": 0,
              "system:index": "409"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0700950291752243, 6.9168463770170225]),
            {
              "CID": 0,
              "system:index": "410"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8634147313236618, 6.94786034506073]),
            {
              "CID": 0,
              "system:index": "411"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7878837254642868, 6.9304791418372425]),
            {
              "CID": 0,
              "system:index": "412"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.85466244065457, 6.86768809053174]),
            {
              "CID": 0,
              "system:index": "413"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.9542260392873825, 6.848258708863722]),
            {
              "CID": 0,
              "system:index": "414"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6730447038381637, 6.832578279907753]),
            {
              "CID": 0,
              "system:index": "415"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4913574276083299, 6.881789679960816]),
            {
              "CID": 0,
              "system:index": "416"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3863006649130174, 6.81122865421331]),
            {
              "CID": 0,
              "system:index": "417"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4165130672567674, 6.761113790084907]),
            {
              "CID": 0,
              "system:index": "418"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5895477352255174, 6.763841263696528]),
            {
              "CID": 0,
              "system:index": "419"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5157333431356736, 6.7198588781227775]),
            {
              "CID": 0,
              "system:index": "420"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8528762874716111, 6.795663192806292]),
            {
              "CID": 0,
              "system:index": "421"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6417327938192674, 6.772480705022038]),
            {
              "CID": 0,
              "system:index": "422"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6310897884481736, 6.814072019418459]),
            {
              "CID": 0,
              "system:index": "423"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.0049837479045598, 6.83535403834419]),
            {
              "CID": 0,
              "system:index": "424"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.090471113627216, 6.801605480985584]),
            {
              "CID": 0,
              "system:index": "425"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6478564849401889, 6.693943316183823]),
            {
              "CID": 0,
              "system:index": "426"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6447236648107944, 6.693453153860153]),
            {
              "CID": 0,
              "system:index": "427"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.64982522635712, 6.692190450824911]),
            {
              "CID": 0,
              "system:index": "428"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6479154935385165, 6.691385942124847]),
            {
              "CID": 0,
              "system:index": "429"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.646402727654117, 6.691300696092207]),
            {
              "CID": 0,
              "system:index": "430"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6446539273764071, 6.691279384581716]),
            {
              "CID": 0,
              "system:index": "431"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6583868360099951, 6.6971235369632645]),
            {
              "CID": 0,
              "system:index": "432"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6568204259452979, 6.703389005933827]),
            {
              "CID": 0,
              "system:index": "433"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6435381269035498, 6.69524814284804]),
            {
              "CID": 0,
              "system:index": "434"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6459199285087744, 6.702067723256897]),
            {
              "CID": 0,
              "system:index": "435"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.646155962902085, 6.7053069905439395]),
            {
              "CID": 0,
              "system:index": "436"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6390105580864112, 6.695397322189217]),
            {
              "CID": 0,
              "system:index": "437"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6377445554313819, 6.68853502531892]),
            {
              "CID": 0,
              "system:index": "438"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6393753385124366, 6.686361234154246]),
            {
              "CID": 0,
              "system:index": "439"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6553827619133155, 6.68689402700095]),
            {
              "CID": 0,
              "system:index": "440"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6565843915519873, 6.692733398520711]),
            {
              "CID": 0,
              "system:index": "441"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6638370847282569, 6.698466144232259]),
            {
              "CID": 0,
              "system:index": "442"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6638800000724951, 6.691369463976362]),
            {
              "CID": 0,
              "system:index": "443"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6636439656791846, 6.686616974793148]),
            {
              "CID": 0,
              "system:index": "444"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6962600911796533, 6.7402563121575705]),
            {
              "CID": 0,
              "system:index": "445"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6986418927848779, 6.741066067386862]),
            {
              "CID": 0,
              "system:index": "446"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.701259728783413, 6.743069140299877]),
            {
              "CID": 0,
              "system:index": "447"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.702482816094204, 6.745583624363471]),
            {
              "CID": 0,
              "system:index": "448"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7030407155693017, 6.742131532732889]),
            {
              "CID": 0,
              "system:index": "449"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7016888822257958, 6.740554643188881]),
            {
              "CID": 0,
              "system:index": "450"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.703856107109829, 6.740874283375853]),
            {
              "CID": 0,
              "system:index": "451"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6836858953178369, 6.736228825269639]),
            {
              "CID": 0,
              "system:index": "452"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6839433873832665, 6.733075002667758]),
            {
              "CID": 0,
              "system:index": "453"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6864110196769677, 6.730112947911715]),
            {
              "CID": 0,
              "system:index": "454"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.695466157311245, 6.735760015099013]),
            {
              "CID": 0,
              "system:index": "455"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6723905088903468, 6.754352172676901]),
            {
              "CID": 0,
              "system:index": "456"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6487012388708155, 6.755545460964499]),
            {
              "CID": 0,
              "system:index": "457"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.646469640970425, 6.749067574886582]),
            {
              "CID": 0,
              "system:index": "458"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6349683287145655, 6.747192381157855]),
            {
              "CID": 0,
              "system:index": "459"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.632908392191128, 6.740884858068462]),
            {
              "CID": 0,
              "system:index": "460"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6406331541540187, 6.737304876015801]),
            {
              "CID": 0,
              "system:index": "461"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6303334715368312, 6.735429636774673]),
            {
              "CID": 0,
              "system:index": "462"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6207204344274562, 6.731338180546081]),
            {
              "CID": 0,
              "system:index": "463"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6341100218298, 6.754693112488008]),
            {
              "CID": 0,
              "system:index": "464"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6428952370731587, 6.7622978202805974]),
            {
              "CID": 0,
              "system:index": "465"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6383247529117817, 6.762585483534453]),
            {
              "CID": 0,
              "system:index": "466"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.63911868678019, 6.761360249968261]),
            {
              "CID": 0,
              "system:index": "467"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6426162873356098, 6.759112204651429]),
            {
              "CID": 0,
              "system:index": "468"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5914055713337283, 6.679149304118543]),
            {
              "CID": 0,
              "system:index": "469"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6010615237873416, 6.676677101257265]),
            {
              "CID": 0,
              "system:index": "470"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5964266666096072, 6.663250264318833]),
            {
              "CID": 0,
              "system:index": "471"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6008469470661502, 6.656344890712266]),
            {
              "CID": 0,
              "system:index": "472"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6042801746052127, 6.65451196658757]),
            {
              "CID": 0,
              "system:index": "473"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6071984180134158, 6.653105299247507]),
            {
              "CID": 0,
              "system:index": "474"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6113182910602908, 6.6478196041516115]),
            {
              "CID": 0,
              "system:index": "475"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.615609825484119, 6.648288498756335]),
            {
              "CID": 0,
              "system:index": "476"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.614236534468494, 6.650846097816972]),
            {
              "CID": 0,
              "system:index": "477"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6145369418781619, 6.661246863465009]),
            {
              "CID": 0,
              "system:index": "478"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6091296085041384, 6.665807785431583]),
            {
              "CID": 0,
              "system:index": "479"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6229054340046267, 6.6658504106704415]),
            {
              "CID": 0,
              "system:index": "480"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6222187884968142, 6.6752278730845225]),
            {
              "CID": 0,
              "system:index": "481"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6269394763630252, 6.677188592529072]),
            {
              "CID": 0,
              "system:index": "482"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5905472644489627, 6.64965255324501]),
            {
              "CID": 0,
              "system:index": "483"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5918776401203494, 6.645603005420873]),
            {
              "CID": 0,
              "system:index": "484"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.589216888777576, 6.6594139576451425]),
            {
              "CID": 0,
              "system:index": "485"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5722224124592166, 6.658348311579349]),
            {
              "CID": 0,
              "system:index": "486"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5625664600056033, 6.66218462658542]),
            {
              "CID": 0,
              "system:index": "487"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5659567722004275, 6.651144483507007]),
            {
              "CID": 0,
              "system:index": "488"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.574110687605701, 6.669899791459253]),
            {
              "CID": 0,
              "system:index": "489"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5805050738972048, 6.671349035399888]),
            {
              "CID": 0,
              "system:index": "490"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5836808093708377, 6.673267145787183]),
            {
              "CID": 0,
              "system:index": "491"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5736386188190798, 6.66798166788773]),
            {
              "CID": 0,
              "system:index": "492"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5659996875446658, 6.6663192880532876]),
            {
              "CID": 0,
              "system:index": "493"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5736386188190798, 6.665637284439071]),
            {
              "CID": 0,
              "system:index": "494"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5790888675373416, 6.666745539830353]),
            {
              "CID": 0,
              "system:index": "495"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5775439151447634, 6.664827403914983]),
            {
              "CID": 0,
              "system:index": "496"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5900825821745834, 6.642689687385642]),
            {
              "CID": 0,
              "system:index": "497"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.591842111288353, 6.636892347616404]),
            {
              "CID": 0,
              "system:index": "498"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5805124604094467, 6.631350709032202]),
            {
              "CID": 0,
              "system:index": "499"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.583001550375267, 6.620736165805221]),
            {
              "CID": 0,
              "system:index": "500"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6081499420988998, 6.6195425482676455]),
            {
              "CID": 0,
              "system:index": "501"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.615788873373314, 6.626832096064799]),
            {
              "CID": 0,
              "system:index": "502"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6226124131072006, 6.639322123331603]),
            {
              "CID": 0,
              "system:index": "503"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6282343232024155, 6.627983067449625]),
            {
              "CID": 0,
              "system:index": "504"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6243290268767319, 6.6185620745601375]),
            {
              "CID": 0,
              "system:index": "505"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6205524765837631, 6.609140901925264]),
            {
              "CID": 0,
              "system:index": "506"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6120123230803451, 6.606455195390061]),
            {
              "CID": 0,
              "system:index": "507"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6175484024870834, 6.599673482488467]),
            {
              "CID": 0,
              "system:index": "508"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6219257675993881, 6.592340902707181]),
            {
              "CID": 0,
              "system:index": "509"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6431322005194038, 6.621037344702282]),
            {
              "CID": 0,
              "system:index": "510"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.647895803729853, 6.624873950609236]),
            {
              "CID": 0,
              "system:index": "511"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6366519835394233, 6.628923668918279]),
            {
              "CID": 0,
              "system:index": "512"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6218891051214546, 6.631097714499797]),
            {
              "CID": 0,
              "system:index": "513"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6255369093817085, 6.622145700583659]),
            {
              "CID": 0,
              "system:index": "514"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6271247771185249, 6.617499115044764]),
            {
              "CID": 0,
              "system:index": "515"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6400852110784858, 6.6139608600382624]),
            {
              "CID": 0,
              "system:index": "516"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6515865233343452, 6.5992107531181645]),
            {
              "CID": 0,
              "system:index": "517"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6531743910711616, 6.596311836489993]),
            {
              "CID": 0,
              "system:index": "518"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6495265868109077, 6.61805329726812]),
            {
              "CID": 0,
              "system:index": "519"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6519727614324897, 6.626877499613284]),
            {
              "CID": 0,
              "system:index": "520"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6321029570501655, 6.63195027877686]),
            {
              "CID": 0,
              "system:index": "521"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.608542433063349, 6.618564849537673]),
            {
              "CID": 0,
              "system:index": "522"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5753930953414041, 6.618522220202119]),
            {
              "CID": 0,
              "system:index": "523"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6036313918501932, 6.586762344147129]),
            {
              "CID": 0,
              "system:index": "524"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5713590529830057, 6.58740182482523]),
            {
              "CID": 0,
              "system:index": "525"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5546649840743143, 6.6028770057289154]),
            {
              "CID": 0,
              "system:index": "526"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5616601851851541, 6.622188329606322]),
            {
              "CID": 0,
              "system:index": "527"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.564835920658787, 6.628156356423955]),
            {
              "CID": 0,
              "system:index": "528"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5657371428877909, 6.6155381575692696]),
            {
              "CID": 0,
              "system:index": "529"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5803283599288065, 6.619843727892925]),
            {
              "CID": 0,
              "system:index": "530"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.557068243351658, 6.604155924670323]),
            {
              "CID": 0,
              "system:index": "531"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5575832274825174, 6.627133271240738]),
            {
              "CID": 0,
              "system:index": "532"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4124887964130028, 6.495366885869539]),
            {
              "CID": 0,
              "system:index": "533"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4317148706317528, 6.498436947209528]),
            {
              "CID": 0,
              "system:index": "534"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4269083520770653, 6.522655662316153]),
            {
              "CID": 0,
              "system:index": "535"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3912027856708153, 6.519585749034458]),
            {
              "CID": 0,
              "system:index": "536"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3884562036395653, 6.507305908015482]),
            {
              "CID": 0,
              "system:index": "537"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4052790185809716, 6.497072477815687]),
            {
              "CID": 0,
              "system:index": "538"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4214151880145653, 6.51037589643665]),
            {
              "CID": 0,
              "system:index": "539"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.441734001352466, 6.482318928441031]),
            {
              "CID": 0,
              "system:index": "540"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4298893663427004, 6.477415192435796]),
            {
              "CID": 0,
              "system:index": "541"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.420533821298755, 6.475709533958258]),
            {
              "CID": 0,
              "system:index": "542"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4101053926488527, 6.477756323439561]),
            {
              "CID": 0,
              "system:index": "543"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4470125886937746, 6.489013517120348]),
            {
              "CID": 0,
              "system:index": "544"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.446755096628345, 6.493789220282379]),
            {
              "CID": 0,
              "system:index": "545"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.447227165414966, 6.496560812948468]),
            {
              "CID": 0,
              "system:index": "546"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4707447740575441, 6.481082338639059]),
            {
              "CID": 0,
              "system:index": "547"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4731480333348879, 6.494258260114205]),
            {
              "CID": 0,
              "system:index": "548"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4233846912969672, 6.499442027985049]),
            {
              "CID": 0,
              "system:index": "549"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4099951038946235, 6.530823738068742]),
            {
              "CID": 0,
              "system:index": "550"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.3821859608282172, 6.498759795113983]),
            {
              "CID": 0,
              "system:index": "551"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.38303812496821354, 6.6826771833836]),
            {
              "CID": 0,
              "system:index": "552"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.42080362789790104, 6.705863965086641]),
            {
              "CID": 0,
              "system:index": "553"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4788251733080573, 6.72427621190288]),
            {
              "CID": 0,
              "system:index": "554"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.548519692351026, 6.747461011645867]),
            {
              "CID": 0,
              "system:index": "555"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.5739255761400885, 6.771326557848656]),
            {
              "CID": 0,
              "system:index": "556"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6096311425463385, 6.726321974078123]),
            {
              "CID": 0,
              "system:index": "557"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6120344018236823, 6.769962844107421]),
            {
              "CID": 0,
              "system:index": "558"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.6532331322924323, 6.819394999381708]),
            {
              "CID": 0,
              "system:index": "559"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7043882226244635, 6.8408708315302835]),
            {
              "CID": 0,
              "system:index": "560"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.731167397429151, 6.863709148951384]),
            {
              "CID": 0,
              "system:index": "561"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7445569848314948, 6.877002590601468]),
            {
              "CID": 0,
              "system:index": "562"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.37891825192133854, 6.733482073833438]),
            {
              "CID": 0,
              "system:index": "563"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.36449869625727604, 6.690519894547652]),
            {
              "CID": 0,
              "system:index": "564"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.36724527828852604, 6.660512320102176]),
            {
              "CID": 0,
              "system:index": "565"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3772016381518073, 6.6414156348142415]),
            {
              "CID": 0,
              "system:index": "566"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3861280297533698, 6.62675160546746]),
            {
              "CID": 0,
              "system:index": "567"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3964277123705573, 6.635618280039985]),
            {
              "CID": 0,
              "system:index": "568"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.40089090817133854, 6.656761244083612]),
            {
              "CID": 0,
              "system:index": "569"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4314466332689948, 6.690178909723211]),
            {
              "CID": 0,
              "system:index": "570"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.4575391625658698, 6.696316600131946]),
            {
              "CID": 0,
              "system:index": "571"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.48328836910883854, 6.705182016647109]),
            {
              "CID": 0,
              "system:index": "572"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.49564798824946354, 6.714729207921294]),
            {
              "CID": 0,
              "system:index": "573"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.36518534176508854, 6.6247054271505155]),
            {
              "CID": 0,
              "system:index": "574"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3318830346361823, 6.632890089470832]),
            {
              "CID": 0,
              "system:index": "575"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2096601342455573, 6.6424386903331625]),
            {
              "CID": 0,
              "system:index": "576"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2178998803393073, 6.681995202553324]),
            {
              "CID": 0,
              "system:index": "577"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.20176371090571354, 6.718138873714433]),
            {
              "CID": 0,
              "system:index": "578"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27180155270258854, 6.6826771833836]),
            {
              "CID": 0,
              "system:index": "579"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2433057641283698, 6.63016188379547]),
            {
              "CID": 0,
              "system:index": "580"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.18803080074946354, 6.5882138243737485]),
            {
              "CID": 0,
              "system:index": "581"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.16537149899165104, 6.539440484354001]),
            {
              "CID": 0,
              "system:index": "582"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.048298439909619795, 6.574571413576297]),
            {
              "CID": 0,
              "system:index": "583"
            }),
        ee.Feature(
            ee.Geometry.Point([0.0038866186841302053, 6.654374180784444]),
            {
              "CID": 0,
              "system:index": "584"
            }),
        ee.Feature(
            ee.Geometry.Point([0.013842978547411455, 6.703477141373682]),
            {
              "CID": 0,
              "system:index": "585"
            }),
        ee.Feature(
            ee.Geometry.Point([0.049891867707567705, 6.605948397437092]),
            {
              "CID": 0,
              "system:index": "586"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.015339455534619795, 6.5524016978583415]),
            {
              "CID": 0,
              "system:index": "587"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.12897928707758854, 6.577982051404871]),
            {
              "CID": 0,
              "system:index": "588"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1485486840502448, 6.624023365825427]),
            {
              "CID": 0,
              "system:index": "589"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.19077738278071354, 6.639028496999027]),
            {
              "CID": 0,
              "system:index": "590"
            }),
        ee.Feature(
            ee.Geometry.Point([0.020709433625536455, 6.5721839531728286]),
            {
              "CID": 0,
              "system:index": "591"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.055508217741651045, 6.535006307801868]),
            {
              "CID": 0,
              "system:index": "592"
            }),
        ee.Feature(
            ee.Geometry.Point([0.0035432959302239553, 6.469171230657753]),
            {
              "CID": 0,
              "system:index": "593"
            }),
        ee.Feature(
            ee.Geometry.Point([0.0035432959302239553, 6.5210213397588275]),
            {
              "CID": 0,
              "system:index": "594"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.0033231591479010447, 6.437785682216691]),
            {
              "CID": 0,
              "system:index": "595"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.06786783688227604, 6.525114541563395]),
            {
              "CID": 0,
              "system:index": "596"
            }),
        ee.Feature(
            ee.Geometry.Point([0.06259480960209896, 6.516928104528502]),
            {
              "CID": 0,
              "system:index": "597"
            }),
        ee.Feature(
            ee.Geometry.Point([0.07083455569584896, 6.585144314626126]),
            {
              "CID": 0,
              "system:index": "598"
            }),
        ee.Feature(
            ee.Geometry.Point([0.09830037600834896, 6.5292077099216055]),
            {
              "CID": 0,
              "system:index": "599"
            }),
        ee.Feature(
            ee.Geometry.Point([0.037875571320848955, 6.482816513814392]),
            {
              "CID": 0,
              "system:index": "600"
            }),
        ee.Feature(
            ee.Geometry.Point([0.0021700049145989553, 6.3982098353542405]),
            {
              "CID": 0,
              "system:index": "601"
            }),
        ee.Feature(
            ee.Geometry.Point([0.09967366702397396, 6.634254186644079]),
            {
              "CID": 0,
              "system:index": "602"
            }),
        ee.Feature(
            ee.Geometry.Point([0.20541707522709896, 6.796554566110308]),
            {
              "CID": 0,
              "system:index": "603"
            }),
        ee.Feature(
            ee.Geometry.Point([0.14499227053959896, 6.838825555739749]),
            {
              "CID": 0,
              "system:index": "604"
            }),
        ee.Feature(
            ee.Geometry.Point([0.048861899445848955, 6.847006606339532]),
            {
              "CID": 0,
              "system:index": "605"
            }),
        ee.Feature(
            ee.Geometry.Point([0.12576619632084896, 6.837462033681397]),
            {
              "CID": 0,
              "system:index": "606"
            }),
        ee.Feature(
            ee.Geometry.Point([0.06671468264897396, 6.836098507732196]),
            {
              "CID": 0,
              "system:index": "607"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.0033231591479010447, 6.814281564011493]),
            {
              "CID": 0,
              "system:index": "608"
            }),
        ee.Feature(
            ee.Geometry.Point([0.17520467288334896, 6.781554288011845]),
            {
              "CID": 0,
              "system:index": "609"
            }),
        ee.Feature(
            ee.Geometry.Point([0.20953694827397396, 6.7501885622463]),
            {
              "CID": 0,
              "system:index": "610"
            }),
        ee.Feature(
            ee.Geometry.Point([0.22189656741459896, 6.691542847592614]),
            {
              "CID": 0,
              "system:index": "611"
            }),
        ee.Feature(
            ee.Geometry.Point([0.16833821780522396, 6.666991383852941]),
            {
              "CID": 0,
              "system:index": "612"
            }),
        ee.Feature(
            ee.Geometry.Point([0.13125936038334896, 6.639710537556239]),
            {
              "CID": 0,
              "system:index": "613"
            }),
        ee.Feature(
            ee.Geometry.Point([0.14087239749272396, 6.926082839699749]),
            {
              "CID": 0,
              "system:index": "614"
            }),
        ee.Feature(
            ee.Geometry.Point([0.16559163577397396, 7.0010566809708665]),
            {
              "CID": 0,
              "system:index": "615"
            }),
        ee.Feature(
            ee.Geometry.Point([0.15323201663334896, 7.08828379649908]),
            {
              "CID": 0,
              "system:index": "616"
            }),
        ee.Feature(
            ee.Geometry.Point([0.12439290530522396, 7.126440479735158]),
            {
              "CID": 0,
              "system:index": "617"
            }),
        ee.Feature(
            ee.Geometry.Point([0.11203328616459896, 6.9806104590248585]),
            {
              "CID": 0,
              "system:index": "618"
            }),
        ee.Feature(
            ee.Geometry.Point([0.09692708499272396, 6.90427002399374]),
            {
              "CID": 0,
              "system:index": "619"
            }),
        ee.Feature(
            ee.Geometry.Point([0.046115317414598955, 6.991515222072125]),
            {
              "CID": 0,
              "system:index": "620"
            }),
        ee.Feature(
            ee.Geometry.Point([0.17657796389897396, 7.069204268813562]),
            {
              "CID": 0,
              "system:index": "621"
            }),
        ee.Feature(
            ee.Geometry.Point([0.18499080075520702, 7.176856935947624]),
            {
              "CID": 0,
              "system:index": "622"
            }),
        ee.Feature(
            ee.Geometry.Point([0.16301814450520702, 7.213643799964119]),
            {
              "CID": 0,
              "system:index": "623"
            }),
        ee.Feature(
            ee.Geometry.Point([0.22618953122395702, 7.2667751000938265]),
            {
              "CID": 0,
              "system:index": "624"
            }),
        ee.Feature(
            ee.Geometry.Point([0.21794978513020702, 7.352589295635684]),
            {
              "CID": 0,
              "system:index": "625"
            }),
        ee.Feature(
            ee.Geometry.Point([0.21245662106770702, 7.426131130077247]),
            {
              "CID": 0,
              "system:index": "626"
            }),
        ee.Feature(
            ee.Geometry.Point([0.14791194333333202, 7.413875008719365]),
            {
              "CID": 0,
              "system:index": "627"
            }),
        ee.Feature(
            ee.Geometry.Point([0.10945979489583202, 7.426131130077247]),
            {
              "CID": 0,
              "system:index": "628"
            }),
        ee.Feature(
            ee.Geometry.Point([0.06551448239583202, 7.550034846302596]),
            {
              "CID": 0,
              "system:index": "629"
            }),
        ee.Feature(
            ee.Geometry.Point([0.10122004880208202, 7.509191362381967]),
            {
              "CID": 0,
              "system:index": "630"
            }),
        ee.Feature(
            ee.Geometry.Point([0.08062068356770702, 7.457450775361638]),
            {
              "CID": 0,
              "system:index": "631"
            }),
        ee.Feature(
            ee.Geometry.Point([0.03667537106770702, 7.515998876796592]),
            {
              "CID": 0,
              "system:index": "632"
            }),
        ee.Feature(
            ee.Geometry.Point([0.02156916989583202, 7.589513213140668]),
            {
              "CID": 0,
              "system:index": "633"
            }),
        ee.Feature(
            ee.Geometry.Point([0.013329423802082019, 7.650765566287868]),
            {
              "CID": 0,
              "system:index": "634"
            }),
        ee.Feature(
            ee.Geometry.Point([0.02156916989583202, 7.703843827324964]),
            {
              "CID": 0,
              "system:index": "635"
            }),
        ee.Feature(
            ee.Geometry.Point([0.03392878903645702, 7.75419397908225]),
            {
              "CID": 0,
              "system:index": "636"
            }),
        ee.Feature(
            ee.Geometry.Point([0.07375422848958202, 7.6344324625647095]),
            {
              "CID": 0,
              "system:index": "637"
            }),
        ee.Feature(
            ee.Geometry.Point([0.20147029294270702, 6.9983305697560265]),
            {
              "CID": 0,
              "system:index": "638"
            }),
        ee.Feature(
            ee.Geometry.Point([0.19185725583333202, 6.90427002399374]),
            {
              "CID": 0,
              "system:index": "639"
            }),
        ee.Feature(
            ee.Geometry.Point([0.17949763669270702, 6.8551875166771845]),
            {
              "CID": 0,
              "system:index": "640"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.11301334963541798, 6.552028425360618]),
            {
              "CID": 0,
              "system:index": "641"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.05121525393229298, 6.4974525855968235]),
            {
              "CID": 0,
              "system:index": "642"
            }),
        ee.Feature(
            ee.Geometry.Point([0.08886042966145702, 6.50291043683581]),
            {
              "CID": 0,
              "system:index": "643"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.03198917971354298, 6.467433346670112]),
            {
              "CID": 0,
              "system:index": "644"
            }),
        ee.Feature(
            ee.Geometry.Point([0.11220637692708202, 6.412848349815951]),
            {
              "CID": 0,
              "system:index": "645"
            }),
        ee.Feature(
            ee.Geometry.Point([0.10671321286458202, 6.381459326595553]),
            {
              "CID": 0,
              "system:index": "646"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.01688297854166798, 6.363716853397156]),
            {
              "CID": 0,
              "system:index": "647"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.03473576174479298, 6.321405409128321]),
            {
              "CID": 0,
              "system:index": "648"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.03885563479166798, 6.302295878895161]),
            {
              "CID": 0,
              "system:index": "649"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.04572208986979298, 6.284550684762805]),
            {
              "CID": 0,
              "system:index": "650"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26680358068713605, 6.09998775501726]),
            {
              "CID": 0,
              "system:index": "651"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26912100927600324, 6.1035188808567]),
            {
              "CID": 0,
              "system:index": "652"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2624476732469505, 6.098462216115143]),
            {
              "CID": 0,
              "system:index": "653"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2653015436387962, 6.095677829728038]),
            {
              "CID": 0,
              "system:index": "654"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26919611112842023, 6.093512185880645]),
            {
              "CID": 0,
              "system:index": "655"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2594470601616683, 6.094824260117766]),
            {
              "CID": 0,
              "system:index": "656"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2557992559014144, 6.100883758798081]),
            {
              "CID": 0,
              "system:index": "657"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2592324834404769, 6.1108689803940965]),
            {
              "CID": 0,
              "system:index": "658"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2708625417290511, 6.11244782316892]),
            {
              "CID": 0,
              "system:index": "659"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2706050496636214, 6.108479371659245]),
            {
              "CID": 0,
              "system:index": "660"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26597019248588705, 6.117269665498413]),
            {
              "CID": 0,
              "system:index": "661"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27056213431938314, 6.119104514432336]),
            {
              "CID": 0,
              "system:index": "662"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27584072166069173, 6.1148374141818485]),
            {
              "CID": 0,
              "system:index": "663"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27760025077446127, 6.1104422653310255]),
            {
              "CID": 0,
              "system:index": "664"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2794026952324691, 6.1042548586965]),
            {
              "CID": 0,
              "system:index": "665"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28129097037895345, 6.100755741928344]),
            {
              "CID": 0,
              "system:index": "666"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28270717673881673, 6.096232459578881]),
            {
              "CID": 0,
              "system:index": "667"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2726649861870589, 6.093245365418187]),
            {
              "CID": 0,
              "system:index": "668"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2736091237603011, 6.089575484120347]),
            {
              "CID": 0,
              "system:index": "669"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2603482823906722, 6.0925199257556555]),
            {
              "CID": 0,
              "system:index": "670"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2557992559014144, 6.0978966905398995]),
            {
              "CID": 0,
              "system:index": "671"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2546834569512191, 6.105791049083796]),
            {
              "CID": 0,
              "system:index": "672"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2555417638359847, 6.111253023659963]),
            {
              "CID": 0,
              "system:index": "673"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25661464744194173, 6.117013639565078]),
            {
              "CID": 0,
              "system:index": "674"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25888916068657064, 6.123243569124243]),
            {
              "CID": 0,
              "system:index": "675"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25773044639213705, 6.127041230263474]),
            {
              "CID": 0,
              "system:index": "676"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25403972678764486, 6.122987546053154]),
            {
              "CID": 0,
              "system:index": "677"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26163574271782064, 6.120854015688703]),
            {
              "CID": 0,
              "system:index": "678"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2836942296562972, 6.113429263626711]),
            {
              "CID": 0,
              "system:index": "679"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28635498099907064, 6.118549793325883]),
            {
              "CID": 0,
              "system:index": "680"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28995986991508627, 6.114453373489014]),
            {
              "CID": 0,
              "system:index": "681"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28948780112846517, 6.106516470790784]),
            {
              "CID": 0,
              "system:index": "682"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2866983037529769, 6.097171257166089]),
            {
              "CID": 0,
              "system:index": "683"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2841662984429183, 6.09025825463314]),
            {
              "CID": 0,
              "system:index": "684"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24515625053032064, 6.113258578459027]),
            {
              "CID": 0,
              "system:index": "685"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24622913413627767, 6.116245561029402]),
            {
              "CID": 0,
              "system:index": "686"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2639960866509261, 6.099475571549697]),
            {
              "CID": 0,
              "system:index": "687"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2654122930107894, 6.10822334151927]),
            {
              "CID": 0,
              "system:index": "688"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2685880284844222, 6.115605494740181]),
            {
              "CID": 0,
              "system:index": "689"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2685880284844222, 6.1108689803940965]),
            {
              "CID": 0,
              "system:index": "690"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2608203511772933, 6.113599948739916]),
            {
              "CID": 0,
              "system:index": "691"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26489730887993, 6.113642620009722]),
            {
              "CID": 0,
              "system:index": "692"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27193542533500814, 6.116928297560042]),
            {
              "CID": 0,
              "system:index": "693"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2711200337944808, 6.123158228114185]),
            {
              "CID": 0,
              "system:index": "694"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2740732625265019, 6.0852652211449545]),
            {
              "CID": 0,
              "system:index": "695"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2788797810811894, 6.084283729138375]),
            {
              "CID": 0,
              "system:index": "696"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28265633137415813, 6.080144373570765]),
            {
              "CID": 0,
              "system:index": "697"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2843300297994511, 6.0777119538921625]),
            {
              "CID": 0,
              "system:index": "698"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28673328907679485, 6.074212664191843]),
            {
              "CID": 0,
              "system:index": "699"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.29158272297572063, 6.073316500975974]),
            {
              "CID": 0,
              "system:index": "700"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2945867970724003, 6.073913943285605]),
            {
              "CID": 0,
              "system:index": "701"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27046837361048626, 6.074767431149426]),
            {
              "CID": 0,
              "system:index": "702"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2707687810201542, 6.079120198204418]),
            {
              "CID": 0,
              "system:index": "703"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2728287175435917, 6.082662129722874]),
            {
              "CID": 0,
              "system:index": "704"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27089752705286907, 6.089447210471341]),
            {
              "CID": 0,
              "system:index": "705"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2602545216817753, 6.089745922748847]),
            {
              "CID": 0,
              "system:index": "706"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25900997669886516, 6.086844139307421]),
            {
              "CID": 0,
              "system:index": "707"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2566496327657597, 6.085137200550089]),
            {
              "CID": 0,
              "system:index": "708"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25639214070033, 6.089575230039179]),
            {
              "CID": 0,
              "system:index": "709"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25377430470179485, 6.093287784229594]),
            {
              "CID": 0,
              "system:index": "710"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25647797138880657, 6.094952024324695]),
            {
              "CID": 0,
              "system:index": "711"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2552763417501347, 6.084923832824214]),
            {
              "CID": 0,
              "system:index": "712"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25240101368616985, 6.08325956165554]),
            {
              "CID": 0,
              "system:index": "713"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2463499501485722, 6.079376242228797]),
            {
              "CID": 0,
              "system:index": "714"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2548471883077519, 6.069774507957612]),
            {
              "CID": 0,
              "system:index": "715"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25167145283411907, 6.0716521939042565]),
            {
              "CID": 0,
              "system:index": "716"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2515856221456425, 6.071097423737543]),
            {
              "CID": 0,
              "system:index": "717"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25283016712855266, 6.071353471577788]),
            {
              "CID": 0,
              "system:index": "718"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2705971196432011, 6.066275166689138]),
            {
              "CID": 0,
              "system:index": "719"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27690567524622844, 6.068707638027213]),
            {
              "CID": 0,
              "system:index": "720"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27368702442835735, 6.0738285944248185]),
            {
              "CID": 0,
              "system:index": "721"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2787939503927128, 6.074298012991753]),
            {
              "CID": 0,
              "system:index": "722"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28900780232142376, 6.070371954195954]),
            {
              "CID": 0,
              "system:index": "723"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2447620824117558, 6.066403191770061]),
            {
              "CID": 0,
              "system:index": "724"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.23883976490687298, 6.073700571108265]),
            {
              "CID": 0,
              "system:index": "725"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24364628346156048, 6.064363072324914]),
            {
              "CID": 0,
              "system:index": "726"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24969734699915813, 6.062997464728324]),
            {
              "CID": 0,
              "system:index": "727"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2575937703390019, 6.069142671676735]),
            {
              "CID": 0,
              "system:index": "728"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25853790791224407, 6.065088549954185]),
            {
              "CID": 0,
              "system:index": "729"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25973953755091594, 6.061205099511387]),
            {
              "CID": 0,
              "system:index": "730"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25557674915980266, 6.058388513570684]),
            {
              "CID": 0,
              "system:index": "731"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24901070149134563, 6.058985972422672]),
            {
              "CID": 0,
              "system:index": "732"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24527706654261516, 6.060308914670138]),
            {
              "CID": 0,
              "system:index": "733"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2556625798482792, 6.062229308935917]),
            {
              "CID": 0,
              "system:index": "734"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25128521473597454, 6.066326127186685]),
            {
              "CID": 0,
              "system:index": "735"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.25261559040736126, 6.062528036318501]),
            {
              "CID": 0,
              "system:index": "736"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2428738072652714, 6.069099996870036]),
            {
              "CID": 0,
              "system:index": "737"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24755157978724407, 6.070337564892147]),
            {
              "CID": 0,
              "system:index": "738"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2477232411641972, 6.066368802213007]),
            {
              "CID": 0,
              "system:index": "739"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24171509297083782, 6.073666182016957]),
            {
              "CID": 0,
              "system:index": "740"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24347462208460735, 6.076482688128729]),
            {
              "CID": 0,
              "system:index": "741"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24433292896937298, 6.072983390437698]),
            {
              "CID": 0,
              "system:index": "742"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.23905434162806438, 6.071020359825443]),
            {
              "CID": 0,
              "system:index": "743"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.23849644215296673, 6.065814026607418]),
            {
              "CID": 0,
              "system:index": "744"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24021305592249798, 6.062186633582049]),
            {
              "CID": 0,
              "system:index": "745"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24137177021693157, 6.060351590172525]),
            {
              "CID": 0,
              "system:index": "746"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24377502949427532, 6.058772594337158]),
            {
              "CID": 0,
              "system:index": "747"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24596371205042766, 6.057278945376997]),
            {
              "CID": 0,
              "system:index": "748"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.24733700306605266, 6.055699940554873]),
            {
              "CID": 0,
              "system:index": "749"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26370079686748094, 6.056100749413519]),
            {
              "CID": 0,
              "system:index": "750"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2660611408005864, 6.057252995775697]),
            {
              "CID": 0,
              "system:index": "751"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2681639926682622, 6.05503385244169]),
            {
              "CID": 0,
              "system:index": "752"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26228459050761765, 6.05025412814482]),
            {
              "CID": 0,
              "system:index": "753"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2658894794236333, 6.050936948489793]),
            {
              "CID": 0,
              "system:index": "754"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.268593146110645, 6.050766243484405]),
            {
              "CID": 0,
              "system:index": "755"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2706101672898442, 6.048675102793779]),
            {
              "CID": 0,
              "system:index": "756"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.27258427312480515, 6.04624254120277]),
            {
              "CID": 0,
              "system:index": "757"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26966602971660203, 6.045474361585338]),
            {
              "CID": 0,
              "system:index": "758"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26554615666972703, 6.044791534343039]),
            {
              "CID": 0,
              "system:index": "759"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26477368047343797, 6.043681938236424]),
            {
              "CID": 0,
              "system:index": "760"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.26228459050761765, 6.0408652609797056]),
            {
              "CID": 0,
              "system:index": "761"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2610829608689458, 6.038219278087027]),
            {
              "CID": 0,
              "system:index": "762"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2603963153611333, 6.036426830718832]),
            {
              "CID": 0,
              "system:index": "763"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.28133900334941453, 6.048675102793779]),
            {
              "CID": 0,
              "system:index": "764"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.019439061604797914, 6.352616814044446]),
            {
              "CID": 0,
              "system:index": "765"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.0015862784016729137, 6.375136489027611]),
            {
              "CID": 0,
              "system:index": "766"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.013945897542297914, 6.417443504435411]),
            {
              "CID": 0,
              "system:index": "767"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.062011083089172914, 6.485673212904275]),
            {
              "CID": 0,
              "system:index": "768"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.08192380281573541, 6.503411428195842]),
            {
              "CID": 0,
              "system:index": "769"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.12312253328448541, 6.522513421460837]),
            {
              "CID": 0,
              "system:index": "770"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1437218985188604, 6.5600330722234785]),
            {
              "CID": 0,
              "system:index": "771"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1897271475422979, 6.56480809816641]),
            {
              "CID": 0,
              "system:index": "772"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1965936026204229, 6.540250334731571]),
            {
              "CID": 0,
              "system:index": "773"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2288659414876104, 6.609145433456551]),
            {
              "CID": 0,
              "system:index": "774"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2625115713704229, 6.613919986789286]),
            {
              "CID": 0,
              "system:index": "775"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2618249258626104, 6.653478793789154]),
            {
              "CID": 0,
              "system:index": "776"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3030236563313604, 6.650068677020765]),
            {
              "CID": 0,
              "system:index": "777"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.2824242910969854, 6.643248372505445]),
            {
              "CID": 0,
              "system:index": "778"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.3016503653157354, 6.639156144402007]),
            {
              "CID": 0,
              "system:index": "779"
            }),
        ee.Feature(
            ee.Geometry.Point([0.019699732340514586, 6.622104828012102]),
            {
              "CID": 0,
              "system:index": "780"
            }),
        ee.Feature(
            ee.Geometry.Point([0.029312769449889586, 6.658934931368801]),
            {
              "CID": 0,
              "system:index": "781"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.07986386629229791, 6.683486799132006]),
            {
              "CID": 0,
              "system:index": "782"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.11213620515948541, 6.7155387704739695]),
            {
              "CID": 0,
              "system:index": "783"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1423486075032354, 6.699854026377763]),
            {
              "CID": 0,
              "system:index": "784"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.12243588777667291, 6.673257003791012]),
            {
              "CID": 0,
              "system:index": "785"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.07025082918292291, 6.650750702267894]),
            {
              "CID": 0,
              "system:index": "786"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.06887753816729791, 6.620740697230163]),
            {
              "CID": 0,
              "system:index": "787"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1264517608613258, 6.734053933468331]),
            {
              "CID": 0,
              "system:index": "788"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.05092075500195081, 6.7149601221319735]),
            {
              "CID": 0,
              "system:index": "789"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.04817417297070081, 6.683590092323908]),
            {
              "CID": 0,
              "system:index": "790"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.04954746398632581, 6.656992185681884]),
            {
              "CID": 0,
              "system:index": "791"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.11340549621288831, 6.657674201314869]),
            {
              "CID": 0,
              "system:index": "792"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1497977081269508, 6.677452242077442]),
            {
              "CID": 0,
              "system:index": "793"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.1642172637910133, 6.75382887459858]),
            {
              "CID": 0,
              "system:index": "794"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.026201516720700813, 6.74360055753467]),
            {
              "CID": 0,
              "system:index": "795"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.07289341125195081, 6.749737573702973]),
            {
              "CID": 0,
              "system:index": "796"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.08593967590038831, 6.723825199265299]),
            {
              "CID": 0,
              "system:index": "797"
            }),
        ee.Feature(
            ee.Geometry.Point([0.05688258972463167, 6.732008203866357]),
            {
              "CID": 0,
              "system:index": "798"
            }),
        ee.Feature(
            ee.Geometry.Point([0.11181423034963167, 6.715642056878529]),
            {
              "CID": 0,
              "system:index": "799"
            }),
        ee.Feature(
            ee.Geometry.Point([0.09258815613088167, 6.694501635806127]),
            {
              "CID": 0,
              "system:index": "800"
            }),
        ee.Feature(
            ee.Geometry.Point([0.07473537292775667, 6.678134229240901]),
            {
              "CID": 0,
              "system:index": "801"
            }),
        ee.Feature(
            ee.Geometry.Point([0.06443569031056917, 6.708822666422018]),
            {
              "CID": 0,
              "system:index": "802"
            }),
        ee.Feature(
            ee.Geometry.Point([0.06512233581838167, 6.777012258898388]),
            {
              "CID": 0,
              "system:index": "803"
            }),
        ee.Feature(
            ee.Geometry.Point([0.02117702331838167, 6.774284859705958]),
            {
              "CID": 0,
              "system:index": "804"
            }),
        ee.Feature(
            ee.Geometry.Point([0.03079006042775667, 6.759965761065631]),
            {
              "CID": 0,
              "system:index": "805"
            }),
        ee.Feature(
            ee.Geometry.Point([0.10906764831838167, 6.7449643456251]),
            {
              "CID": 0,
              "system:index": "806"
            }),
        ee.Feature(
            ee.Geometry.Point([0.08503505554494417, 6.759965761065631]),
            {
              "CID": 0,
              "system:index": "807"
            }),
        ee.Feature(
            ee.Geometry.Point([0.08228847351369417, 6.799512712985606]),
            {
              "CID": 0,
              "system:index": "808"
            }),
        ee.Feature(
            ee.Geometry.Point([0.07130214538869417, 6.739509170235862]),
            {
              "CID": 0,
              "system:index": "809"
            }),
        ee.Feature(
            ee.Geometry.Point([0.16056606140431917, 6.727916718799319]),
            {
              "CID": 0,
              "system:index": "810"
            }),
        ee.Feature(
            ee.Geometry.Point([0.15369960632619417, 6.75382887459858]),
            {
              "CID": 0,
              "system:index": "811"
            }),
        ee.Feature(
            ee.Geometry.Point([0.13653346863088167, 6.722461351764844]),
            {
              "CID": 0,
              "system:index": "812"
            }),
        ee.Feature(
            ee.Geometry.Point([0.16331264343556917, 6.717005923504462]),
            {
              "CID": 0,
              "system:index": "813"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.11203220519726331, 6.7040490362892085]),
            {
              "CID": 0,
              "system:index": "814"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.09074619445507581, 6.684954048584334]),
            {
              "CID": 0,
              "system:index": "815"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.008348733517575813, 6.587421630723258]),
            {
              "CID": 0,
              "system:index": "816"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.0035422149628883126, 6.618797800802778]),
            {
              "CID": 0,
              "system:index": "817"
            }),
        ee.Feature(
            ee.Geometry.Point([0.05344936218556917, 6.552632688166541]),
            {
              "CID": 0,
              "system:index": "818"
            }),
        ee.Feature(
            ee.Geometry.Point([0.02529689636525667, 6.523298948702391]),
            {
              "CID": 0,
              "system:index": "819"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.776962548825213, 6.550415663372513]),
            {
              "CID": 0,
              "system:index": "820"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7827990356416192, 6.56013639125591]),
            {
              "CID": 0,
              "system:index": "821"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7877772155732599, 6.564655612460824]),
            {
              "CID": 0,
              "system:index": "822"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7829706970185724, 6.568918991089433]),
            {
              "CID": 0,
              "system:index": "823"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7706969085664239, 6.560989077640499]),
            {
              "CID": 0,
              "system:index": "824"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7695811096162286, 6.551183096146905]),
            {
              "CID": 0,
              "system:index": "825"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7668345275849786, 6.545555228397114]),
            {
              "CID": 0,
              "system:index": "826"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.762628823849627, 6.547004836763236]),
            {
              "CID": 0,
              "system:index": "827"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7660620513886895, 6.5376249434665175]),
            {
              "CID": 0,
              "system:index": "828"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7774775329560724, 6.540865290150527]),
            {
              "CID": 0,
              "system:index": "829"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7833998504609552, 6.5456404995939845]),
            {
              "CID": 0,
              "system:index": "830"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.783829003903338, 6.552803228127195]),
            {
              "CID": 0,
              "system:index": "831"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7967894378632989, 6.5675547139017265]),
            {
              "CID": 0,
              "system:index": "832"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7926695648164239, 6.569430594070438]),
            {
              "CID": 0,
              "system:index": "833"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7960169616670099, 6.573438132550691]),
            {
              "CID": 0,
              "system:index": "834"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8003943267793145, 6.574376062424834]),
            {
              "CID": 0,
              "system:index": "835"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7996218505830255, 6.570283264536336]),
            {
              "CID": 0,
              "system:index": "836"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7977335754365411, 6.566360968294487]),
            {
              "CID": 0,
              "system:index": "837"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7909529510468927, 6.560733271878365]),
            {
              "CID": 0,
              "system:index": "838"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7805674377412286, 6.551779987488642]),
            {
              "CID": 0,
              "system:index": "839"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7772200408906427, 6.548880794282871]),
            {
              "CID": 0,
              "system:index": "840"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7753317657441583, 6.543167628973801]),
            {
              "CID": 0,
              "system:index": "841"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7761900726289239, 6.539330391708245]),
            {
              "CID": 0,
              "system:index": "842"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7864039245576349, 6.542400183878246]),
            {
              "CID": 0,
              "system:index": "843"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.8088915649384942, 6.528074325845862]),
            {
              "CID": 0,
              "system:index": "844"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7581604355750149, 6.546671028555431]),
            {
              "CID": 0,
              "system:index": "845"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7606547180146661, 6.542570727334699]),
            {
              "CID": 0,
              "system:index": "846"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7604401412934747, 6.546365304179347]),
            {
              "CID": 0,
              "system:index": "847"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7736580673188653, 6.5403536578602015]),
            {
              "CID": 0,
              "system:index": "848"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7721560302705255, 6.550415663372513]),
            {
              "CID": 0,
              "system:index": "849"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.771769792172381, 6.548070722582472]),
            {
              "CID": 0,
              "system:index": "850"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.775589257809588, 6.553101672918042]),
            {
              "CID": 0,
              "system:index": "851"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7758467498750177, 6.559752681906736]),
            {
              "CID": 0,
              "system:index": "852"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7494395421041244, 6.5743312351447925]),
            {
              "CID": 0,
              "system:index": "853"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7425301716817612, 6.576057874305047]),
            {
              "CID": 0,
              "system:index": "854"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.744954888631224, 6.571176372796318]),
            {
              "CID": 0,
              "system:index": "855"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7529156849874252, 6.570877938835309]),
            {
              "CID": 0,
              "system:index": "856"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7568638966573471, 6.5737983206126]),
            {
              "CID": 0,
              "system:index": "857"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7575290844930405, 6.577422128164924]),
            {
              "CID": 0,
              "system:index": "858"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7580869839681381, 6.582943054746728]),
            {
              "CID": 0,
              "system:index": "859"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7400410817159409, 6.57292433954324]),
            {
              "CID": 0,
              "system:index": "860"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7413499997152084, 6.568959430525243]),
            {
              "CID": 0,
              "system:index": "861"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7445901082051987, 6.565911118774189]),
            {
              "CID": 0,
              "system:index": "862"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7468002484334701, 6.564163127390321]),
            {
              "CID": 0,
              "system:index": "863"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7499545262349838, 6.569215232060711]),
            {
              "CID": 0,
              "system:index": "864"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7339041874898666, 6.567381984817294]),
            {
              "CID": 0,
              "system:index": "865"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7483666584981674, 6.56444024838492]),
            {
              "CID": 0,
              "system:index": "866"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7564776585592026, 6.566870379730088]),
            {
              "CID": 0,
              "system:index": "867"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7623570607198471, 6.57128295631042]),
            {
              "CID": 0,
              "system:index": "868"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7415431187642807, 6.561924836780782]),
            {
              "CID": 0,
              "system:index": "869"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7688489980739388, 6.580071028226214]),
            {
              "CID": 0,
              "system:index": "870"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7690206594508919, 6.575978277228275]),
            {
              "CID": 0,
              "system:index": "871"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.76507244778097, 6.57205602589629]),
            {
              "CID": 0,
              "system:index": "872"
            }),
        ee.Feature(
            ee.Geometry.Point([-0.7738271780055794, 6.577171999751674]),
            {
              "CID": 0,
              "system:index": "873"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5873729028027017, 6.1068115198049595]),
            {
              "CID": 0,
              "system:index": "874"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5880595483105142, 6.105275332344321]),
            {
              "CID": 0,
              "system:index": "875"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.581450585297819, 6.113212253446245]),
            {
              "CID": 0,
              "system:index": "876"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5824805535595377, 6.11193211284208]),
            {
              "CID": 0,
              "system:index": "877"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5879737176220377, 6.0909373704136]),
            {
              "CID": 0,
              "system:index": "878"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5456591882030923, 6.110139910851096]),
            {
              "CID": 0,
              "system:index": "879"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5456591882030923, 6.109286479222242]),
            {
              "CID": 0,
              "system:index": "880"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5226565636913736, 6.0892304446625865]),
            {
              "CID": 0,
              "system:index": "881"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5194808282177408, 6.088803712377142]),
            {
              "CID": 0,
              "system:index": "882"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.522570733002897, 6.088718365879367]),
            {
              "CID": 0,
              "system:index": "883"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4981948174755533, 6.106128770366408]),
            {
              "CID": 0,
              "system:index": "884"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4974196590702493, 6.106750179070675]),
            {
              "CID": 0,
              "system:index": "885"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4970602430622537, 6.106419472382192]),
            {
              "CID": 0,
              "system:index": "886"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4969636835377176, 6.10629945780764]),
            {
              "CID": 0,
              "system:index": "887"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4971916713039835, 6.105902076024423]),
            {
              "CID": 0,
              "system:index": "888"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4975135363857706, 6.105382504333061]),
            {
              "CID": 0,
              "system:index": "889"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.497701291016813, 6.104990455864311]),
            {
              "CID": 0,
              "system:index": "890"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4983195341470235, 6.1043583891434565]),
            {
              "CID": 0,
              "system:index": "891"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4989471710565083, 6.104035681984716]),
            {
              "CID": 0,
              "system:index": "892"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4994648373963826, 6.1037903177339405]),
            {
              "CID": 0,
              "system:index": "893"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5002721823098653, 6.10341160312631]),
            {
              "CID": 0,
              "system:index": "894"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4972137286032505, 6.102893345699479]),
            {
              "CID": 0,
              "system:index": "895"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.4943384005392857, 6.103842799635495]),
            {
              "CID": 0,
              "system:index": "896"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5016961285744057, 6.107619262678285]),
            {
              "CID": 0,
              "system:index": "897"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.542323580016971, 6.018711143535264]),
            {
              "CID": 0,
              "system:index": "898"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5555415060423616, 6.010388717296684]),
            {
              "CID": 0,
              "system:index": "899"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5313157942198519, 6.010602114431965]),
            {
              "CID": 0,
              "system:index": "900"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5561205990109994, 6.0013542679894725]),
            {
              "CID": 0,
              "system:index": "901"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5725609297658028, 6.002076096612146]),
            {
              "CID": 0,
              "system:index": "902"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5804788107777656, 5.995247239934207]),
            {
              "CID": 0,
              "system:index": "903"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5964937595829642, 5.997992603881174]),
            {
              "CID": 0,
              "system:index": "904"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6020512966618217, 5.989520477384191]),
            {
              "CID": 0,
              "system:index": "905"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5871811298832572, 5.986682178903981]),
            {
              "CID": 0,
              "system:index": "906"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5901346891975, 6.041947447180144]),
            {
              "CID": 0,
              "system:index": "907"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5910412758445336, 6.042123489586144]),
            {
              "CID": 0,
              "system:index": "908"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5894212215995385, 6.041077903547078]),
            {
              "CID": 0,
              "system:index": "909"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.588756033763845, 6.040768495046265]),
            {
              "CID": 0,
              "system:index": "910"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5882034987067772, 6.040555109770279]),
            {
              "CID": 0,
              "system:index": "911"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5876509636497094, 6.04167004690904]),
            {
              "CID": 0,
              "system:index": "912"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5866209953879906, 6.041920774083337]),
            {
              "CID": 0,
              "system:index": "913"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.588112303600271, 6.042726301027575]),
            {
              "CID": 0,
              "system:index": "914"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5890993565177514, 6.042475574226396]),
            {
              "CID": 0,
              "system:index": "915"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5896250694846703, 6.043345115614245]),
            {
              "CID": 0,
              "system:index": "916"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5916850060081078, 6.04201146260713]),
            {
              "CID": 0,
              "system:index": "917"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5918352097129418, 6.041184596092567]),
            {
              "CID": 0,
              "system:index": "918"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5893729418372704, 6.0398776109616445]),
            {
              "CID": 0,
              "system:index": "919"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.589185187206228, 6.0388426898853576]),
            {
              "CID": 0,
              "system:index": "920"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.5875544041251732, 6.0391467648397406]),
            {
              "CID": 0,
              "system:index": "921"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8241949223696041, 4.924206728702153]),
            {
              "CID": 0,
              "system:index": "922"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8309755467592526, 4.932330499335292]),
            {
              "CID": 0,
              "system:index": "923"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.837756171148901, 4.923180561138307]),
            {
              "CID": 0,
              "system:index": "924"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8390436314760494, 4.908728533288408]),
            {
              "CID": 0,
              "system:index": "925"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8255682133852291, 4.896328614843379]),
            {
              "CID": 0,
              "system:index": "926"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8166418217836666, 4.901288609841111]),
            {
              "CID": 0,
              "system:index": "927"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8127794408022213, 4.89119754682815]),
            {
              "CID": 0,
              "system:index": "928"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.826769843023901, 4.886665070704361]),
            {
              "CID": 0,
              "system:index": "929"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8291731023012447, 4.884783656516061]),
            {
              "CID": 0,
              "system:index": "930"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.8327779912172604, 4.890000291964972]),
            {
              "CID": 0,
              "system:index": "931"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7941541814028072, 4.895644474714824]),
            {
              "CID": 0,
              "system:index": "932"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7601652287660885, 4.885895401901992]),
            {
              "CID": 0,
              "system:index": "933"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7484064244447994, 4.889658218753752]),
            {
              "CID": 0,
              "system:index": "934"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7456598424135494, 4.89119754682815]),
            {
              "CID": 0,
              "system:index": "935"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7610235356508541, 4.912234675352255]),
            {
              "CID": 0,
              "system:index": "936"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7721815251528072, 4.923009533057086]),
            {
              "CID": 0,
              "system:index": "937"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7895193242250729, 4.925147380911328]),
            {
              "CID": 0,
              "system:index": "938"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7795629643617916, 4.942762985281151]),
            {
              "CID": 0,
              "system:index": "939"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7684049748598385, 4.949774891738879]),
            {
              "CID": 0,
              "system:index": "940"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7678041600405026, 4.938145347811063]),
            {
              "CID": 0,
              "system:index": "941"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7535562657533932, 4.930278187710757]),
            {
              "CID": 0,
              "system:index": "942"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7418832921205807, 4.946610991922467]),
            {
              "CID": 0,
              "system:index": "943"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.740853323858862, 4.936862665028381]),
            {
              "CID": 0,
              "system:index": "944"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7363901280580807, 4.929508569216505]),
            {
              "CID": 0,
              "system:index": "945"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7212839268862057, 4.935922029411282]),
            {
              "CID": 0,
              "system:index": "946"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.71218587390769, 4.937119201783364]),
            {
              "CID": 0,
              "system:index": "947"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7148466252504635, 4.933442165483225]),
            {
              "CID": 0,
              "system:index": "948"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7236013554750729, 4.950116934056137]),
            {
              "CID": 0,
              "system:index": "949"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7168046136385273, 4.962275378686837]),
            {
              "CID": 0,
              "system:index": "950"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7202378411775898, 4.970398682107512]),
            {
              "CID": 0,
              "system:index": "951"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7284775872713398, 4.974930586815162]),
            {
              "CID": 0,
              "system:index": "952"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7503644128328633, 4.966636322439165]),
            {
              "CID": 0,
              "system:index": "953"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7630673547273945, 4.96637979713432]),
            {
              "CID": 0,
              "system:index": "954"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7714787621980976, 4.9644130998151095]),
            {
              "CID": 0,
              "system:index": "955"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7743111749178242, 4.960137650633468]),
            {
              "CID": 0,
              "system:index": "956"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7031575341707539, 4.972023330777553]),
            {
              "CID": 0,
              "system:index": "957"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6986943383699726, 4.9832247466745185]),
            {
              "CID": 0,
              "system:index": "958"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7231899990194943, 5.001159820782488]),
            {
              "CID": 0,
              "system:index": "959"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.7258078350180295, 5.0022286189769325]),
            {
              "CID": 0,
              "system:index": "960"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.724970985805383, 5.005798392296301]),
            {
              "CID": 0,
              "system:index": "961"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6562130871397707, 4.991079360270601]),
            {
              "CID": 0,
              "system:index": "962"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6569855633360597, 4.992832215358001]),
            {
              "CID": 0,
              "system:index": "963"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6566422405821535, 4.99697919532352]),
            {
              "CID": 0,
              "system:index": "964"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6568139019591066, 4.988385939668618]),
            {
              "CID": 0,
              "system:index": "965"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6526940289122316, 4.989326500500012]),
            {
              "CID": 0,
              "system:index": "966"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6387951593240846, 4.982348556724309]),
            {
              "CID": 0,
              "system:index": "967"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6411125879129518, 4.980830816774236]),
            {
              "CID": 0,
              "system:index": "968"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6435158471902955, 4.9794199567767885]),
            {
              "CID": 0,
              "system:index": "969"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6457903604349244, 4.97721815406654]),
            {
              "CID": 0,
              "system:index": "970"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6490519265970338, 4.974952213486486]),
            {
              "CID": 0,
              "system:index": "971"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.646198056205188, 4.973541340882567]),
            {
              "CID": 0,
              "system:index": "972"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6511547784647096, 4.9771754005429205]),
            {
              "CID": 0,
              "system:index": "973"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6505754213174928, 4.981429362525167]),
            {
              "CID": 0,
              "system:index": "974"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.658235810264026, 4.9745246766531075]),
            {
              "CID": 0,
              "system:index": "975"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6585147600015748, 4.9728145265430195]),
            {
              "CID": 0,
              "system:index": "976"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.660660527213489, 4.97418264698648]),
            {
              "CID": 0,
              "system:index": "977"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6563046197733033, 4.971208600481792]),
            {
              "CID": 0,
              "system:index": "978"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6665613870462526, 4.968579229614459]),
            {
              "CID": 0,
              "system:index": "979"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6732990960916627, 4.969412933708415]),
            {
              "CID": 0,
              "system:index": "980"
            }),
        ee.Feature(
            ee.Geometry.Point([-1.6961515168985475, 4.981362576391048]),
            {
              "CID": 0,
              "system:index": "981"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.126081596561946, 6.602584382208031]),
            {
              "CID": 0,
              "system:index": "982"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1284204828229325, 6.600644681027378]),
            {
              "CID": 0,
              "system:index": "983"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1302658426251786, 6.599046020495955]),
            {
              "CID": 0,
              "system:index": "984"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1304375040021317, 6.5958273683071855]),
            {
              "CID": 0,
              "system:index": "985"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1291715013471024, 6.594527112155668]),
            {
              "CID": 0,
              "system:index": "986"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.121639858433284, 6.5994296994941655]),
            {
              "CID": 0,
              "system:index": "987"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.125437866398372, 6.59917391352839]),
            {
              "CID": 0,
              "system:index": "988"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.117176662632503, 6.599408384002053]),
            {
              "CID": 0,
              "system:index": "989"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.117841850468196, 6.596914465093765]),
            {
              "CID": 0,
              "system:index": "990"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.121511112400569, 6.596147102903771]),
            {
              "CID": 0,
              "system:index": "991"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1224767076459305, 6.594164745079372]),
            {
              "CID": 0,
              "system:index": "992"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1244722711530106, 6.597276830159226]),
            {
              "CID": 0,
              "system:index": "993"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.12099612826971, 6.602371228603839]),
            {
              "CID": 0,
              "system:index": "994"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.116597305485286, 6.6022859671364635]),
            {
              "CID": 0,
              "system:index": "995"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.114773403355159, 6.601689136453628]),
            {
              "CID": 0,
              "system:index": "996"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.116103779026546, 6.595869999598642]),
            {
              "CID": 0,
              "system:index": "997"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1150308954205888, 6.598321292688368]),
            {
              "CID": 0,
              "system:index": "998"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.113679062077083, 6.600474157483257]),
            {
              "CID": 0,
              "system:index": "999"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.111704956242122, 6.598811549850721]),
            {
              "CID": 0,
              "system:index": "1000"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1129924165692704, 6.597276830159226]),
            {
              "CID": 0,
              "system:index": "1001"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.110031257816829, 6.5968078870829]),
            {
              "CID": 0,
              "system:index": "1002"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.110052715488948, 6.599387068509036]),
            {
              "CID": 0,
              "system:index": "1003"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1088725435223954, 6.596083156000957]),
            {
              "CID": 0,
              "system:index": "1004"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.105889927097835, 6.59834260822726]),
            {
              "CID": 0,
              "system:index": "1005"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.110318688993189, 6.602861481756276]),
            {
              "CID": 0,
              "system:index": "1006"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.112464456205103, 6.605483262111277]),
            {
              "CID": 0,
              "system:index": "1007"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.100255040769312, 6.604055139575194]),
            {
              "CID": 0,
              "system:index": "1008"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1023149772927496, 6.599493645964959]),
            {
              "CID": 0,
              "system:index": "1009"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1078081413552496, 6.601646505663035]),
            {
              "CID": 0,
              "system:index": "1010"
            }),
        ee.Feature(
            ee.Geometry.Point([-2.1055121704385016, 6.606740859140343]),
            {
              "CID": 0,
              "system:index": "1011"
            })])

## All points data

In [8]:
galamsey_pts = ee.FeatureCollection("projects/servir-wa/services/IllegalMining/2019_galamsey_pts")

pprint(galamsey_pts.aggregate_histogram(OLD_LABEL).getInfo())

{'0': 2000}


In [9]:
galamsey_pts = galamsey_pts.map(set_class_label)

pprint(galamsey_pts.aggregate_histogram(OUTPUT_BANDS[0]).getInfo())

{'1': 2000}


In [10]:
# old labels still exists

pprint(galamsey_pts.first().getInfo()), pprint(user_added_galamsey.first().getInfo()), pprint(user_added_non_galamsey.first().getInfo())

{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-2.0554618144929204, 6.530161515380249],
              'type': 'Point'},
 'id': '0',
 'properties': {'CID': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-0.5952137390609913, 6.8783537421613605],
              'type': 'Point'},
 'id': '0',
 'properties': {'CID': 0},
 'type': 'Feature'}


(None, None, None)

In [11]:
user_added_galamsey = user_added_galamsey.map(set_label)
user_added_non_galamsey = user_added_non_galamsey.map(set_label)

pprint(galamsey_pts.first().getInfo()), pprint(user_added_galamsey.first().getInfo()), pprint(user_added_non_galamsey.first().getInfo())

{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-2.0554618144929204, 6.530161515380249],
              'type': 'Point'},
 'id': '0',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-0.5952137390609913, 6.8783537421613605],
              'type': 'Point'},
 'id': '0',
 'properties': {'mining': 0},
 'type': 'Feature'}


(None, None, None)

In [12]:
all_points = galamsey_pts.merge(user_added_galamsey).merge(user_added_non_galamsey)

map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=galamsey_pts.merge(user_added_galamsey).getMapId({'color': 'green'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="galamsey_pts",
  ).add_to(map)

folium.TileLayer(
    tiles=user_added_non_galamsey.getMapId({'color': 'red'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="non_galamsey_pts",
  ).add_to(map)

roi_outline = ee.Image().byte()\
    .paint(featureCollection=REGION,
           color=1,
           width=3).getMapId()

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)


map.add_child(folium.LayerControl())

## Sentinel-2 surface reflectance

We will use a cloud-freee composite of Sentinel-2 surface reflectance data for predictors.  See [the Code Editor example](https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2%20Cloud%20And%20Shadow) for details.

In [13]:
def get_s2_composite(roi, start, end):
  s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
  s2sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  s2c = s2c.filterBounds(roi.buffer(100, 1000)).filterDate(start, end)
  s2sr = s2sr.filterBounds(roi.buffer(100, 1000)).filterDate(start, end)

  def index_join(collection_a, collection_b, property_name):
    joined = ee.ImageCollection(
        ee.Join.saveFirst(property_name).apply(
            primary=collection_a,
            secondary=collection_b,
            condition=ee.Filter.equals(
                leftField='system:index',
                rightField='system:index')))
    return joined.map(
        lambda image: image.addBands(ee.Image(image.get(property_name))))

  def mask_image(image):
    prob = image.select('probability')
    return image.select('B.*').divide(10000).updateMask(prob.lt(50))

  with_cloud_probability = index_join(s2sr, s2c, 'cloud_probability')
  masked = ee.ImageCollection(with_cloud_probability.map(mask_image))
  return masked.select(S2_BANDS).median().float().unmask(0)

image = get_s2_composite(REGION, START, END)

In [14]:
vis_params = {
  'min': 0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}


map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="image",
  ).add_to(map)

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)

map.add_child(folium.LayerControl())


# Generate training data

There are a number of ways to generate training data in Earth Engine.  This notebook demonstrates using `image.sample` method to generate training data.

Divide data into training (60%), validation (20%) and testing (20%) datasets. Also get the distribution of the galamsey and non-galamsey points in each of the dataset.

In [15]:
all_points = all_points.randomColumn("random", 100)

pprint(all_points.first().getInfo())

pprint(("Sample size:", all_points.size().getInfo()))

training_sample_locations = all_points.filter(ee.Filter.gt("random", VALIDATION_RATIO + TEST_RATIO)) # > 0.4
training_sample_size = training_sample_locations.size().getInfo()
pprint(("Training sample size:", training_sample_size))

validation_sample_locations = all_points.filter(ee.Filter.lte("random", VALIDATION_RATIO)) # <= 0.2
validation_sample_size = validation_sample_locations.size().getInfo()
pprint(("Validation sample size:", validation_sample_size))

test_sample_locations = all_points.filter(ee.Filter.And(ee.Filter.gt("random", VALIDATION_RATIO),
                                                        ee.Filter.lte("random", VALIDATION_RATIO + TEST_RATIO))) # > 0.2 and <= 0.4
test_sample_size = validation_sample_locations.size().getInfo()
pprint(("Test sample size:", test_sample_size))

pprint(("training_sample_locations", training_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))
pprint(("validation_sample_locations", validation_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))
pprint(("test_sample_locations", test_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))


{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '1_1_00000000000000000000',
 'properties': {'mining': 1, 'random': 0.4438691942788049},
 'type': 'Feature'}
('Sample size:', 3132)
('Training sample size:', 1901)
('Validation sample size:', 589)
('Test sample size:', 589)
('training_sample_locations', {'0': 629, '1': 1272})
('validation_sample_locations', {'0': 189, '1': 400})
('test_sample_locations', {'0': 194, '1': 448})


In [16]:
map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=training_sample_locations.getMapId({'color': 'green'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="training_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=validation_sample_locations.getMapId({'color': 'red'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="validation_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=test_sample_locations.getMapId({'color': 'blue'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="test_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)

map.add_child(folium.LayerControl())


Paint the label data into an image and add that as a band to the S2 Image

In [17]:
label_img = ee.Image().byte().paint(featureCollection=all_points, color=OUTPUT_BANDS[0]).rename(OUTPUT_BANDS[0])
image = image.addBands(label_img)

## Export Image before sampling

If you try to sample without exporting you will run into Computational Time Out. This is because there are a lot of processing going on with the S2 data using cloud score etc. So better to export the image to avoid the Computational Time out issue.

In [18]:
def export_image_to_asset(image: ee.Image, start_training: bool, **kwargs: dict) -> ee.batch.Task:
    asset_id = kwargs.get("asset_id", "")
    print(f"Exporting image to {asset_id}..")

    training_task = ee.batch.Export.image.toAsset(
        image=image,
        description=kwargs.get("description", "myExportImageTask"),
        assetId=asset_id,
        region=kwargs.get("region", None),
        scale=kwargs.get("scale", SCALE),
        maxPixels=kwargs.get("max_pixels", 1E13),
    )
    if start_training: training_task.start()
    return training_task

In [19]:
# Change False to True if you're actually exporting
export_image = False
training_task = export_image_to_asset(image, export_image, **{"asset_id": "projects/servir-wa/tf_training_aug_2023/s2_2019_demo", "region": REGION})


Exporting image to projects/servir-wa/tf_training_aug_2023/s2_2019_demo..


In [20]:
#@title Don't run unless you are exporting the image

# Print all tasks.
print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while training_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

[<Task CSFPVKGAD2LD6XO3A2T7HWNN EXPORT_FEATURES: Validation (COMPLETED)>, <Task JLJM3CPASJYEEW6GZXTFZPIX EXPORT_FEATURES: Testing (COMPLETED)>, <Task Y3CM4IYANUH65PIXEH236OR2 EXPORT_FEATURES: Training (COMPLETED)>, <Task HKDQVBODZCUVOBZQWD7PR3YV EXPORT_IMAGE: s1_asc_2019_demo_30_m (COMPLETED)>, <Task L56A5GPEHFPKYTC3B45RZA3M EXPORT_IMAGE: s1_asc_2019_demo (RUNNING)>, <Task O2FGUD43WF3JC3KWTLSP5UCL EXPORT_IMAGE: s1_asc_2019_demo (CANCELLED)>, <Task SPFFSQQJDKLHQRUC63Q7M6KI EXPORT_IMAGE: myExportImageTask (COMPLETED)>, <Task 657WZ3FSURXG65JX6YPNLEFG INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/projects/servir-wa/services/IllegalMining/tf_training_prediction" (COMPLETED)>, <Task FRSK7TBRIYAUM4RR3X5CSS5K EXPORT_IMAGE: Image Export (COMPLETED)>, <Task 7TMQAWNVOOAOM3NJGECIDIAI EXPORT_IMAGE: Image Export (COMPLETED)>, <Task JIL5H7CVLRIING42EFUKC7QC EXPORT_IMAGE: Image Export (FAILED)>, <Task BV56T4GVD64CWAYNES2LSMRO EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task AGZHEB

Finally assemble all the data for the sampling purpose. Here we are adding already exported Sentinel-2 and Sentinel-1 dataset.

In [21]:
s2 = ee.Image("projects/servir-wa/tf_training_aug_2023/s2_2019_demo")
# s1 = ee.Image("projects/servir-wa/tf_training_aug_2023/s1_asc_2019_demo")
s1 = ee.Image("projects/servir-wa/tf_training_aug_2023/s1_asc_2019_demo_30_m")
s1 = s1.select(['VV', 'VH', 'ratio', 'ndratio'], S1_BANDS)
image = s1.addBands(s2)
pprint(image.bandNames().getInfo())

['VV',
 'VH',
 's1_ratio',
 's1_ndratio',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'mining']


In [22]:
def sample_image(image: ee.Image, region: ee.FeatureCollection, **kwargs: dict) -> ee.FeatureCollection:
    sample = image.sample(region=ee.FeatureCollection(region),
                          scale=kwargs.get("seed", SCALE),
                          seed=kwargs.get("seed", 100),
                          geometries=kwargs.get("geometries", False))
    return sample

In [23]:
training_samples = sample_image(image, training_sample_locations)
validation_samples = sample_image(image, validation_sample_locations)
test_samples = sample_image(image, test_sample_locations)


In [24]:
# sanity check before exporting them
example_samples = sample_image(image, ee.FeatureCollection(ee.Feature(training_sample_locations.first())), **{"scale": SCALE})
pprint(example_samples.first().getInfo())

{'geometry': None,
 'id': '0',
 'properties': {'B1': 0.07715000212192535,
                'B11': 0.2533000111579895,
                'B12': 0.16979999840259552,
                'B2': 0.08049999922513962,
                'B3': 0.11225000023841858,
                'B4': 0.12084999680519104,
                'B5': 0.1732500046491623,
                'B6': 0.243599995970726,
                'B7': 0.26205000281333923,
                'B8': 0.25804999470710754,
                'B8A': 0.28334999084472656,
                'B9': 0.30869999527931213,
                'VH': -14.61683173873444,
                'VV': -8.161584434856472,
                'mining': 1,
                's1_ndratio': -0.28339315844804536,
                's1_ratio': 0.5583689120008384},
 'type': 'Feature'}


In [25]:
def export_collection_to_cloud_storage(collection: ee.FeatureCollection, start_training: bool, **kwargs: dict) -> ee.batch.Task:
    description = kwargs.get("description", "myExportTableTask")
    bucket = kwargs.get("bucket", "myBucket")
    file_prefix = kwargs.get("file_prefix") if kwargs.get("file_prefix") is not None else description
    print(f"Exporting training data to gs://{bucket}/{file_prefix}..")
    training_task = ee.batch.Export.table.toCloudStorage(
        collection=collection,
        description=description,
        fileNamePrefix=file_prefix,
        bucket=bucket,
        fileFormat=kwargs.get("file_format", "TFRecord"),
        selectors=kwargs.get("selectors", collection.first().propertyNames().getInfo()),
    )
    if start_training: training_task.start()
    return training_task

In [26]:
# Names for output files.
train_file_prefix = "illegal_mining_training"
test_file_prefix = "illegal_mining_testing"
validate_file_prefix = "illegal_mining_validation"


Finally, let's export the samples in TFRecord format for using with the DL.

In [27]:
# Change start_training=True if you're actually exporting
start_training = False

kwargs = { "bucket": BUCKET, "selectors": BANDS }
training_task = export_collection_to_cloud_storage(training_samples, start_training=start_training, **{**kwargs, "file_prefix": train_file_prefix, "description": "Training"})
testing_task = export_collection_to_cloud_storage(test_samples, start_training=start_training, **{**kwargs, "file_prefix": test_file_prefix, "description": "Testing"})
validation_task = export_collection_to_cloud_storage(validation_samples, start_training=start_training, **{**kwargs, "file_prefix": validate_file_prefix, "description": "Validation"})


Exporting training data to gs://wa-tf-training/illegal_mining_training..
Exporting training data to gs://wa-tf-training/illegal_mining_testing..
Exporting training data to gs://wa-tf-training/illegal_mining_validation..


In [28]:
#@title Don't run unless you are exporting the training data

print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while training_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

while testing_task.active():
  print('Polling for task (id: {}).'.format(testing_task.id))
  time.sleep(30)
print('Done with testing export.')


while validation_task.active():
  print('Polling for task (id: {}).'.format(validation_task.id))
  time.sleep(30)
print('Done with validation export.')



[<Task CSFPVKGAD2LD6XO3A2T7HWNN EXPORT_FEATURES: Validation (COMPLETED)>, <Task JLJM3CPASJYEEW6GZXTFZPIX EXPORT_FEATURES: Testing (COMPLETED)>, <Task Y3CM4IYANUH65PIXEH236OR2 EXPORT_FEATURES: Training (COMPLETED)>, <Task HKDQVBODZCUVOBZQWD7PR3YV EXPORT_IMAGE: s1_asc_2019_demo_30_m (COMPLETED)>, <Task L56A5GPEHFPKYTC3B45RZA3M EXPORT_IMAGE: s1_asc_2019_demo (RUNNING)>, <Task O2FGUD43WF3JC3KWTLSP5UCL EXPORT_IMAGE: s1_asc_2019_demo (CANCELLED)>, <Task SPFFSQQJDKLHQRUC63Q7M6KI EXPORT_IMAGE: myExportImageTask (COMPLETED)>, <Task 657WZ3FSURXG65JX6YPNLEFG INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/projects/servir-wa/services/IllegalMining/tf_training_prediction" (COMPLETED)>, <Task FRSK7TBRIYAUM4RR3X5CSS5K EXPORT_IMAGE: Image Export (COMPLETED)>, <Task 7TMQAWNVOOAOM3NJGECIDIAI EXPORT_IMAGE: Image Export (COMPLETED)>, <Task JIL5H7CVLRIING42EFUKC7QC EXPORT_IMAGE: Image Export (FAILED)>, <Task BV56T4GVD64CWAYNES2LSMRO EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task AGZHEB

### Check existence of the exported files

If you've seen the status of the export tasks change to `COMPLETED`, then check for the existince of the files in the output Cloud Storage bucket.

In [29]:
file_name_suffix = '.tfrecord.gz'
train_file_path = 'gs://' + BUCKET + '/' + train_file_prefix + file_name_suffix
test_file_path = 'gs://' + BUCKET + '/' + test_file_prefix + file_name_suffix
validate_file_path = 'gs://' + BUCKET + '/' + validate_file_prefix + file_name_suffix

print('Found training file.' if tf.io.gfile.exists(train_file_path) else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(test_file_path) else 'No testing file found.')
print('Found validation file.' if tf.io.gfile.exists(validate_file_path) else 'No validation file found.')

Found training file.
Found testing file.
Found validation file.


## Define the structure of your data

For parsing the exported TFRecord files, `FEATURES_DICT` is a mapping between feature names (recall that `BANDS` contains the band and label names) and `float32` [`tf.io.FixedLenFeature`](https://www.tensorflow.org/api_docs/python/tf/io/FixedLenFeature) objects.  This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors.  Specifically, **all numeric data exported from Earth Engine is exported as `float32`**.

(Note: *features* in the TensorFlow context (i.e. [`tf.train.Feature`](https://www.tensorflow.org/api_docs/python/tf/train/Feature)) are not to be confused with Earth Engine features (i.e. [`ee.Feature`](https://developers.google.com/earth-engine/api_docs#eefeature)), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)

In [30]:
COLUMNS = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
  for k in BANDS
]

FEATURES_DICT = dict(zip(BANDS, COLUMNS))

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

In [31]:
def create_tfrecord_from_file(filename: str) -> tf.data.TFRecordDataset:
    return tf.data.TFRecordDataset(filename, compression_type="GZIP")

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/data) explain more about reading data into a `Dataset`).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.

In [32]:
train_dataset = tf.data.Dataset.list_files(train_file_path).interleave(create_tfrecord_from_file)
test_dataset = tf.data.Dataset.list_files(test_file_path).interleave(create_tfrecord_from_file)
validate_dataset = tf.data.Dataset.list_files(validate_file_path).interleave(create_tfrecord_from_file)

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized [`Example` proto](https://www.tensorflow.org/api_docs/python/tf/train/Example) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.  ([These TensorFlow docs](https://www.tensorflow.org/tutorials/load_data/tfrecord) explain more about reading `Example` protos from TFRecord files).

In [33]:
def create_tfrecord_from_file(filename: str) -> tf.data.TFRecordDataset:
    return tf.data.TFRecordDataset(filename, compression_type="GZIP")

In [34]:
def parse_tfrecord_dnn(example_proto: tf.Tensor) -> Tuple:
    parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
    label = parsed_features.pop(OUTPUT_BANDS[0])
    label = tf.cast(label, tf.int32)
    return parsed_features, label

In [35]:
def to_tuple_dnn(dataset: dict, label: tf.Tensor, depth: int = 1) -> Tuple:
    return tf.transpose(list(dataset.values())), tf.one_hot(indices=label, depth=depth)

In [36]:
parser = partial(parse_tfrecord_dnn)
tupler = partial(to_tuple_dnn, depth=N_CLASSES)

In [37]:
train_dataset = train_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)


In [38]:
pprint(iter(train_dataset).next())

({'B1': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04895], dtype=float32)>,
  'B11': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.02475], dtype=float32)>,
  'B12': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0203], dtype=float32)>,
  'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0496], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.05795], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04185], dtype=float32)>,
  'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04425], dtype=float32)>,
  'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0384], dtype=float32)>,
  'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03915], dtype=float32)>,
  'B8': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03395], dtype=float32)>,
  'B8A': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03505], dtype=float32)>,
  'B9': <tf.Tensor: shape=(1,), dtype=float

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values.  The second element of the tuple is a class label.

## Create additional features

Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands.  Here are some helper functions for that.

In [41]:
def normalized_difference(a, b):
    nd = (a - b) / (a + b)
    nd_inf = (a - b) / (a + b + 0.000001)
    return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    features["NDVI"] = normalized_difference(NIR, RED)
    return features, label

def add_EVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    BLUE = features["B2"]
    EVI = 2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))
    features["EVI"] = EVI
    return features, label

def add_MNDWI(features, label):
    GREEN = features["B3"]
    NIR = features["B5"]
    MNDWI = normalized_difference(GREEN, NIR)
    features["MNDWI"] = MNDWI
    return features, label

def add_SAVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    SAVI = (NIR - RED) * (1 + 0.5)/(NIR + RED + 0.5)
    features["SAVI"] = SAVI
    return features, label

def add_IBI(features, label):
    #Add Index-Based Built-Up Index (IBI)
    RED = features["B4"]
    GREEN = features["B3"]
    SWIR1 = features["B11"]
    NIR = features["B5"]

    ibiA = (2 * SWIR1) / (SWIR1 + NIR)
    ibiB = (NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))
    IBI = normalized_difference(ibiA, ibiB)
    features["IBI"] = IBI
    return features, label


In [43]:
train_dataset = train_dataset.map(add_NDVI)
train_dataset = train_dataset.map(add_EVI)
train_dataset = train_dataset.map(add_MNDWI)
train_dataset = train_dataset.map(add_SAVI)
train_dataset = train_dataset.map(add_IBI)


train_dataset = train_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(512)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat()

In [44]:
validate_dataset = validate_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.map(add_NDVI)
validate_dataset = validate_dataset.map(add_EVI)
validate_dataset = validate_dataset.map(add_MNDWI)
validate_dataset = validate_dataset.map(add_SAVI)
validate_dataset = validate_dataset.map(add_IBI)

validate_dataset = validate_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.shuffle(512)
validate_dataset = validate_dataset.batch(1)
validate_dataset = validate_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.repeat()

# Model setup

The basic workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).

Here we'll create a `Sequential` neural network model using Keras.  This simple model is inspired by examples in:

* [The TensorFlow Get Started tutorial](https://www.tensorflow.org/tutorials/)
* [The TensorFlow Keras guide](https://www.tensorflow.org/guide/keras#build_a_simple_model)
* [The Keras `Sequential` model examples](https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification)

Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

We will look at both Sequential and Functional based approach to making models.

## 1.   Sequential Model with keras

In [45]:
# Define the layers in the model.
model1 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  # 2 class = sigmoid, multi-class = softmax
  tf.keras.layers.Dense(N_CLASSES, activation="sigmoid")
])

# Compile the model with the specified loss function.
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [46]:
# Fit the model to the training data.
history1 = model1.fit(
            x=train_dataset,
            epochs=EPOCHS,
            steps_per_epoch=(training_sample_size // BATCH_SIZE),
            validation_data=validate_dataset,
            validation_steps=(validation_sample_size // BATCH_SIZE)
        )

Epoch 1/25
59/59 [==============================] - 11s 24ms/step - loss: 0.6726 - accuracy: 0.6483 - val_loss: 0.5854 - val_accuracy: 0.7778
Epoch 2/25
59/59 [==============================] - 1s 22ms/step - loss: 0.6355 - accuracy: 0.6752 - val_loss: 0.5319 - val_accuracy: 0.7778
Epoch 3/25
59/59 [==============================] - 1s 22ms/step - loss: 0.5979 - accuracy: 0.7250 - val_loss: 0.5737 - val_accuracy: 0.7778
Epoch 4/25
59/59 [==============================] - 2s 33ms/step - loss: 0.5410 - accuracy: 0.7533 - val_loss: 0.4579 - val_accuracy: 0.7778
Epoch 5/25
59/59 [==============================] - 2s 27ms/step - loss: 0.4713 - accuracy: 0.7988 - val_loss: 0.3266 - val_accuracy: 0.8333
Epoch 6/25
59/59 [==============================] - 2s 33ms/step - loss: 0.4107 - accuracy: 0.8283 - val_loss: 0.1976 - val_accuracy: 0.9444
Epoch 7/25
59/59 [==============================] - 1s 19ms/step - loss: 0.3779 - accuracy: 0.8459 - val_loss: 0.1966 - val_accuracy: 0.9444
Epoch 8/25
5

In [50]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 128)            2816      
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 dense_1 (Dense)             (None, 1, 64)             8256      
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 dense_2 (Dense)             (None, 1, 32)             2080      
                                                                 
 dropout_2 (Dropout)         (None, 1, 32)             0         
                                                                 
 dense_3 (Dense)             (None, 1, 2)              6

## 2. Functional Model with keras

In [56]:
inputs = keras.Input(shape=(None, len(INPUT_BANDS) + 5), name="input_layer")
y = keras.layers.Conv1D(128, 3, activation="relu", padding="same", name="conv1")(inputs)
y = keras.layers.MaxPooling1D(2, padding="same")(y)
y = keras.layers.Conv1D(64, 3, activation="relu", padding="same", name="conv2")(y)
y = keras.layers.MaxPooling1D(2, padding="same")(y)
y = keras.layers.Conv1D(len(INPUT_BANDS) + 5, 2, activation="relu", padding="same", name="conv4")(y)

all_inputs = keras.layers.concatenate([inputs, y])

x = keras.layers.Dense(128, activation="relu")(all_inputs)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
output = keras.layers.Dense(N_CLASSES, activation="sigmoid")(x)

model2 = keras.models.Model(inputs=inputs, outputs=output)

# Compile the model with the specified loss function.
model2.compile(optimizer=tf.keras.optimizers.Adam(),
               loss='binary_crossentropy',
               metrics=['accuracy'])


In [57]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, None, 21)]   0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, None, 128)    8192        ['input_layer[0][0]']            
                                                                                                  
 max_pooling1d_4 (MaxPooling1D)  (None, None, 128)   0           ['conv1[0][0]']                  
                                                                                                  
 conv2 (Conv1D)                 (None, None, 64)     24640       ['max_pooling1d_4[0][0]']        
                                                                                            

In [58]:
# Fit the model to the training data.
history2 = model2.fit(
            x=train_dataset,
            epochs=EPOCHS,
            steps_per_epoch=(training_sample_size // BATCH_SIZE),
            validation_data=validate_dataset,
            validation_steps=(validation_sample_size // BATCH_SIZE)
        )

Epoch 1/25
59/59 [==============================] - 13s 31ms/step - loss: 0.7147 - accuracy: 0.6298 - val_loss: 0.6391 - val_accuracy: 0.6667
Epoch 2/25
59/59 [==============================] - 2s 24ms/step - loss: 0.6520 - accuracy: 0.6768 - val_loss: 0.6339 - val_accuracy: 0.6667
Epoch 3/25
59/59 [==============================] - 1s 22ms/step - loss: 0.6116 - accuracy: 0.7159 - val_loss: 0.6631 - val_accuracy: 0.7778
Epoch 4/25
59/59 [==============================] - 3s 43ms/step - loss: 0.5224 - accuracy: 0.7822 - val_loss: 0.5511 - val_accuracy: 0.7222
Epoch 5/25
59/59 [==============================] - 1s 23ms/step - loss: 0.4045 - accuracy: 0.8432 - val_loss: 0.8022 - val_accuracy: 0.6111
Epoch 6/25
59/59 [==============================] - 1s 23ms/step - loss: 0.3785 - accuracy: 0.8502 - val_loss: 0.1110 - val_accuracy: 1.0000
Epoch 7/25
59/59 [==============================] - 1s 21ms/step - loss: 0.3380 - accuracy: 0.8609 - val_loss: 0.0725 - val_accuracy: 1.0000
Epoch 8/25
5

## Check model accuracy on the test set

Now that we have a trained model, we can evaluate it using the test dataset.  To do that, read and prepare the test dataset in the same way as the training dataset.  Here we specify a batch size of 1 so that each example in the test set is used exactly once to compute model accuracy.

In [53]:
test_dataset = test_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(add_NDVI)
test_dataset = test_dataset.map(add_EVI)
test_dataset = test_dataset.map(add_MNDWI)
test_dataset = test_dataset.map(add_SAVI)
test_dataset = test_dataset.map(add_IBI)

test_dataset = test_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(1)

In [54]:
model1.evaluate(test_dataset)

642/642 [==============================] - 7s 10ms/step - loss: 0.2569 - accuracy: 0.8863


[0.25685566663742065, 0.8862928152084351]

In [59]:
model2.evaluate(test_dataset)

642/642 [==============================] - 4s 6ms/step - loss: 0.2922 - accuracy: 0.8551


[0.29222628474235535, 0.855140209197998]

# Inferences

## 1.   Use the trained model to classify an image from Earth Engine



Now it's time to classify the image that was exported from Earth Engine.  If the exported image is large, it will be split into multiple TFRecord files in its destination folder.  There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image.  Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

### Export the imagery

You also need to export imagery using TFRecord format.  Specifically, export whatever imagery you want to be classified by the trained model into the output Cloud Storage bucket.

In [60]:
IMAGE_FILE_PREFIX

'WA_image'

In [70]:
def export_image_to_cloud_storage(image: ee.Image, start_training: bool, region: ee.Geometry, **kwargs: dict) -> ee.batch.Task:

    print(f"Exporting image to BUCKET {BUCKET}..")

    # Specify patch and file dimensions.
    image_export_options = {
    "patchDimensions": [256, 256],
    "maxFileSize": 104857600,
    "compressed": True
    }

    training_task = ee.batch.Export.image.toCloudStorage(
        image=image,
        description=kwargs.get("description", "myExportImageTask"),
        fileNamePrefix=kwargs.get("file_name_prefix", IMAGE_FILE_PREFIX),
        bucket=BUCKET,
        fileFormat="TFRecord",
        region=region.getInfo()['coordinates'],
        scale=kwargs.get("scale", SCALE),
        maxPixels=kwargs.get("max_pixels", 1E13),
        formatOptions=image_export_options,
    )
    if start_training: training_task.start()
    return training_task

In [71]:
# Getting a smaller area for prediction because this might take long to complete

EXPORT_REGION = ee.Geometry.Polygon(
        [[[-2.2499489720464894, 6.806096384151279],
          [-2.2499489720464894, 5.675769064032216],
          [-1.3435769017339894, 5.675769064032216],
          [-1.3435769017339894, 6.806096384151279]]])



In [77]:
start_cloud_export = False
export_task = export_image_to_cloud_storage(image, start_training=start_cloud_export, region=EXPORT_REGION)

Exporting image to BUCKET wa-tf-training..


In [78]:
#@title Don't run unless you are exporting the image

# Print all tasks.
print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while export_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

[<Task 33JLI3SHTCWHCJNII27FZUOE EXPORT_IMAGE: myExportImageTask (READY)>, <Task CZLFHG3IVPDT3KKUK4SEIN5W EXPORT_IMAGE: myExportImageTask (CANCEL_REQUESTED)>, <Task Y2O2N4F5IEK5VEQI4M4BJHJQ EXPORT_IMAGE: myExportImageTask (CANCEL_REQUESTED)>, <Task CSFPVKGAD2LD6XO3A2T7HWNN EXPORT_FEATURES: Validation (COMPLETED)>, <Task JLJM3CPASJYEEW6GZXTFZPIX EXPORT_FEATURES: Testing (COMPLETED)>, <Task Y3CM4IYANUH65PIXEH236OR2 EXPORT_FEATURES: Training (COMPLETED)>, <Task HKDQVBODZCUVOBZQWD7PR3YV EXPORT_IMAGE: s1_asc_2019_demo_30_m (COMPLETED)>, <Task L56A5GPEHFPKYTC3B45RZA3M EXPORT_IMAGE: s1_asc_2019_demo (RUNNING)>, <Task O2FGUD43WF3JC3KWTLSP5UCL EXPORT_IMAGE: s1_asc_2019_demo (CANCELLED)>, <Task SPFFSQQJDKLHQRUC63Q7M6KI EXPORT_IMAGE: myExportImageTask (COMPLETED)>, <Task 657WZ3FSURXG65JX6YPNLEFG INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/projects/servir-wa/services/IllegalMining/tf_training_prediction" (COMPLETED)>, <Task FRSK7TBRIYAUM4RR3X5CSS5K EXPORT_IMAGE: Image Export (CO

### Find the image files and JSON mixer file in Cloud Storage

Use `gsutil` to locate the files of interest in the output Cloud Storage bucket.  Check to make sure your image export task finished before running the following.


In [79]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
pprint(json_file)

['gs://wa-tf-training/WA_image00000.tfrecord.gz',
 'gs://wa-tf-training/WA_image00001.tfrecord.gz',
 'gs://wa-tf-training/WA_image00002.tfrecord.gz',
 'gs://wa-tf-training/WA_image00003.tfrecord.gz',
 'gs://wa-tf-training/WA_image00004.tfrecord.gz',
 'gs://wa-tf-training/WA_image00005.tfrecord.gz',
 'gs://wa-tf-training/WA_image00006.tfrecord.gz',
 'gs://wa-tf-training/WA_image00007.tfrecord.gz',
 'gs://wa-tf-training/WA_image00008.tfrecord.gz']
'gs://wa-tf-training/WA_imagemixer.json'


### Read the JSON mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [80]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 13,
 'projection': {'affine': {'doubleMatrix': [0.0002694945852358564,
                                            0.0,
                                            -2.2500102921341663,
                                            0.0,
                                            -0.0002694945852358564,
                                            6.806355244716791]},
                'crs': 'EPSG:4326'},
 'totalPatches': 208}


### Read the image files into a dataset

You can feed the list of files directly to the `TFRecordDataset` constructor to make a combined dataset on which to perform inference.  The input needs to be preprocessed differently than the training and testing.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.

In [81]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(INPUT_BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_NDVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_EVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_MNDWI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_SAVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_IBI(features, None)[0]
)


# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

### Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [82]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model1.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

208/208 [==============================] - 2889s 14s/step
[[0.8816099 0.1042067]]


In [86]:
pprint(len(predictions)), pprint(predictions[3045])

13631488
array([[0.67055327, 0.31307673]], dtype=float32)


(None, None)

### Write the predictions to a TFRecord file

Now that there's a list of class probabilities in `predictions`, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability.  We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  The records are written as serialized `tf.train.Example` protos.  This might take a while.

In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1).numpy()[0])
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print(f"Done with patch {cur_patch} of {patches} ...")
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          "prediction": tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          "nonMiningProb": tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          "miningProb": tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], []]
    cur_patch += 1

writer.close()

Done with patch 1 of 208 ...
Done with patch 2 of 208 ...
Done with patch 3 of 208 ...
Done with patch 4 of 208 ...
Done with patch 5 of 208 ...
Done with patch 6 of 208 ...
Done with patch 7 of 208 ...
Done with patch 8 of 208 ...
Done with patch 9 of 208 ...
Done with patch 10 of 208 ...
Done with patch 11 of 208 ...
Done with patch 12 of 208 ...
Done with patch 13 of 208 ...
Done with patch 14 of 208 ...
Done with patch 15 of 208 ...
Done with patch 16 of 208 ...
Done with patch 17 of 208 ...
Done with patch 18 of 208 ...
Done with patch 19 of 208 ...
Done with patch 20 of 208 ...
Done with patch 21 of 208 ...
Done with patch 22 of 208 ...
Done with patch 23 of 208 ...
Done with patch 24 of 208 ...
Done with patch 25 of 208 ...
Done with patch 26 of 208 ...
Done with patch 27 of 208 ...
Done with patch 28 of 208 ...
Done with patch 29 of 208 ...
Done with patch 30 of 208 ...
Done with patch 31 of 208 ...
Done with patch 32 of 208 ...
Done with patch 33 of 208 ...
Done with patch 34 

### Upload the classifications to an Earth Engine asset

#### Verify the existence of the predictions file

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket.  Use the `gsutil` command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

### Upload the classified image to Earth Engine

Upload the image to Earth Engine directly from the Cloud Storage bucket with the [`earthengine` command](https://developers.google.com/earth-engine/command_line#upload).  Provide both the image TFRecord file and the JSON file as arguments to `earthengine upload`.

In [ ]:
OUTPUT_ASSET_ID = "projects/servir-wa/tf_training_aug_2023/tf_training_prediction_example"
print('Uploading to ' + OUTPUT_ASSET_ID)

In [ ]:
# Start the upload.
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

### Check the status of the asset ingestion

You can also use the Earth Engine API to check the status of your asset upload.  It might take a while.  The upload of the image is an asset ingestion task.

In [ ]:
ee.batch.Task.list()

[]

### View the ingested asset

Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel.  Also display the winning class using the same color palette.

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 1,
}
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[COORDS[1], COORDS[0]])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)

map.add_child(folium.LayerControl())
map

## 2. Using the AI Platform

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs. You can still use this notebook for training, but may need to set up an alternative VM (learn more) for production use. Alternatively, you can package your code for running large training jobs on Google's AI Platform as described here. The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
MODEL_DIR = 'gs://' + BUCKET + '/model'
model1.save(MODEL_DIR)

### Prepare the model for making predictions in Earth Engine

Before we can use the model in Earth Engine, it needs to be hosted by AI Platform.  But before we can host the model on AI Platform we need to *EEify* (a new word!) it.  The EEification process merely appends some extra operations to the input and outputs of the model in order to accomdate the interchange format between pixels from Earth Engine (float32) and inputs to AI Platform (base64).  (See [this doc](https://cloud.google.com/ml-engine/docs/online-predict#binary_data_in_prediction_input) for details.)  

## `earthengine model prepare`
The EEification process is handled for you using the Earth Engine command `earthengine model prepare`.  To use that command, we need to specify the input and output model directories and the name of the input and output nodes in the TensorFlow computation graph.  We can do all that programmatically:

In [ ]:
# ee.Authenticate()

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: OUTPUT_BANDS[0]}) + "'"

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + BUCKET + '/eeified'

# You need to set the project before using the model prepare command.
# if you get config not found; rerun the ee.Authenticate() (above cell)
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

### Perform inference using the trained model in Earth Engine

Before it's possible to get predictions from the trained and EEified model, it needs to be deployed on AI Platform.  The first step is to create the model.  The second step is to create a version.  See [this guide](https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models) for details.  Note that models and versions can be monitored from the [AI Platform models page](http://console.cloud.google.com/ai-platform/models) of the Cloud Console.  

In [ ]:
MODEL_NAME = 'wa_tf_training_model'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=2.8 \
  --framework "TENSORFLOW" \
  --python-version=3.7

There is now a trained model, prepared for serving to Earth Engine, hosted and versioned on AI Platform.  We can now connect Earth Engine directly to the trained model for inference.  You do that with the `ee.Model.fromAiPlatformPredictor` command.

#### `ee.Model.fromAiPlatformPredictor`
For this command to work, we need to know a lot about the model.  To connect to the model, you need to know the name and version.

##### Inputs
You need to be able to recreate the imagery on which it was trained in order to perform inference.  Specifically, you need to create an array-valued input from the scaled data and use that for input.  (Recall that the new input node is named `array`, which is convenient because the array image has one band, named `array` by default.)  The inputs will be provided as 16x16 patches (`inputTileSize`), at 30-meter resolution (`proj`), but 8 pixels will be thrown out (`inputOverlapSize`) to minimize boundary effects.

##### Outputs
The output (which you also need to know), is a single float band named `mining`.

In [ ]:
image.select(INPUT_BANDS).bandNames().getInfo()

In [ ]:
# Load the trained model and use it for prediction.
model = ee.Model.fromAiPlatformPredictor(
    projectName = PROJECT,
    modelName = MODEL_NAME,
    version = VERSION_NAME,
    inputTileSize = [16, 16],
    inputOverlapSize = [8, 8],
    proj = ee.Projection('EPSG:4326').atScale(SCALE),
    fixInputProj = True,
    outputBands = {OUTPUT_BANDS[0]: {
        'type': ee.PixelType.float()
      }
    }
)
predictions = model.predictImage(image.select(INPUT_BANDS).toArray())


vis_params = {
  'min': 0,
  'max': 0.4,
  'bands': ['B4', 'B3', 'B2'],
}


map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=image.clip(EXPORT_REGION).getMapId(vis_params)["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="Image",
  ).add_to(map)

folium.TileLayer(
    tiles=predictions.getMapId({'min': 0, 'max': 1})["tile_fetcher"].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='predictions',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## 3. Using Vertex AI Platform (coming soon ...)